# Phosphosite Data Processing, 1 of 2.
## Phospho.ELM Data.

A phosphosite data set has been downloaded as a tab-delimited file from http://phospho.elm.eu.org/dataset.html:

- phosphoELM_vertebrate_2015-04.dump
    - The full list of vertebrate phosphosites available on the website (46,248 rows long on 2020_01_17)

In this Jupyter notebook, it will be imported as a pandas dataframe, and will have the following changes made to it:

- Remove any rows with information about non-human proteins.
- Translate important substrates using UniProt API
- Translate kinases using "human_kinase_dataframe.csv".
- Generate a phosphosite ID column.
- Remove unnecessary columns.

It will then be exported as "phosphosites_2.csv" for subsequent incorporation into data tables with data from phosphosite.org, in Jupyter notebook "Phosphosite-data-processing-2-of-2-Phosphosite.org-downloaded-files.ipynb".

Import required packages

In [ ]:
import pandas as pd # v 0.25.1
import urllib.parse # version not available
import urllib.request # v 3.7

Read in phosphosite data from phospho.ELM (and table of human kinases, for translating kinase aliases)

In [ ]:
# This table was downloaded from phospho.ELM

phosphosite_2_df = pd.read_table( "phosphoELM_vertebrate_2015-04.dump" )

# This table was generated by script "SDP_kinase_dataframe_script.ipynb"

kinases_df = pd.read_csv( "human_kinase_dataframe.csv" )

Remove any rows where species is not "Homo sapiens"

In [ ]:
phosphosite_2_df = phosphosite_2_df.drop( phosphosite_2_df[ phosphosite_2_df.species != "Homo sapiens" ].index )

phosphosite_2_df = phosphosite_2_df.reset_index( drop = True )

Remove NaNs from kinase column and make kinases uppercase

In [ ]:
kinases_minus_NaNs = []

for i in phosphosite_2_df.kinases:
    if type( i ) != float:
        kinases_minus_NaNs.append( i.upper() )
    else:
        kinases_minus_NaNs.append( "" )

kinases_minus_NaNs = pd.Series( kinases_minus_NaNs )
phosphosite_2_df = phosphosite_2_df.assign( kinases = kinases_minus_NaNs )

Translate important substrate IDs to gene symbol and entry name (minus "_HUMAN", which will be added later). This will allow the phosphopeptides in the quantitative phosphoproteomics file to be queried in the database

In [ ]:
# A subset of substrate IDs was chosen out of necessity. There are 5,374 unique substrate IDs
# in this table, but only some will be added to the tables generated from phosphosite.org
# data, as most of the phosphosites here are already there.
# There are 296 unique substrate IDs to query. I attempted to query all 5,374 on two occasions
# and the connection timed out before 2,000 were returned. Hence I have selected only the
# important IDs.

substrates = ['O00141','O00161','O00257','O14745','O14757','O14964','O14994','O15273','O15357','O15492','O15530','O43521','O43586','O43914','O60381','O60506','O60674','O75582','O75676','O75914','O95997','O96017','P00519','P00533','P01100','P01236','P01350','P02686','P03372','P04049','P04150','P04629','P04637','P05106','P05198','P05412','P05787','P06213','P06239','P06401','P06733','P07101','P07203','P07550','P07947','P07948','P08047','P08069','P08100','P08631','P08670','P09038','P09619','P09769','P0C0S8','P10242','P10244','P10721','P10747','P10828','P10912','P11137','P11171','P11274','P11362','P11388','P11831','P11912','P12272','P12318','P12931','P13569','P13693','P14317','P14598','P14859','P15172','P15260','P15336','P15941','P16104','P16220','P16234','P16410','P16885','P17252','P17275','P17480','P17542','P17600','P17655','P17676','P18031','P18206','P18507','P19105','P19174','P19235','P19419','P19429','P19793','P20138','P20700','P20936','P20963','P22001','P22314','P22607','P22681','P23396','P23443','P23458','P23528','P23634','P24844','P24928','P25963','P27448','P27708','P27986','P28562','P28749','P29320','P29322','P29350','P29353','P29597','P30260','P30304','P30305','P30307','P30419','P30443','P30530','P31152','P31946','P33241','P33778','P33981','P33991','P35222','P35568','P35611','P35612','P35637','P36888','P36956','P38398','P38432','P38936','P39880','P40259','P41212','P41743','P42229','P42702','P42768','P43354','P46527','P46695','P48751','P49023','P49407','P49715','P49768','P49841','P49917','P50549','P50613','P51674','P51692','P51812','P51813','P52735','P52799','P52926','P53355','P53667','P53671','P53778','P54259','P55211','P56270','P56945','P58012','P60953','P61925','P61978','P62158','P62993','P68400','P68431','P78314','P78347','P78527','P78536','P84022','P84243','Q00169','Q00535','Q01892','Q02078','Q02241','Q02363','Q02763','Q03721','Q04206','Q04912','Q05513','Q05655','Q05682','Q06124','Q06413','Q07666','Q07820','Q08379','Q12772','Q12888','Q12972','Q13094','Q13131','Q13153','Q13158','Q13177','Q13224','Q13322','Q13415','Q13418','Q13424','Q13444','Q13480','Q13485','Q13507','Q13541','Q13651','Q13765','Q13813','Q14005','Q14118','Q14191','Q14247','Q14289','Q14847','Q14934','Q15139','Q15149','Q15418','Q15648','Q15653','Q15788','Q15796','Q15831','Q16760','Q16821','Q16828','Q7KZI7','Q86WB0','Q86WV1','Q8TDC3','Q8WXE1','Q92731','Q92918','Q92934','Q92974','Q99683','Q9BUB5','Q9BXS5','Q9BXW9','Q9H0H5','Q9H1A4','Q9H1D0','Q9H8V3','Q9HBA0','Q9NQ66','Q9NQS7','Q9NQU5','Q9NRQ2','Q9NRY4','Q9NXH3','Q9NYV6','Q9UBS0','Q9UEW8','Q9UJX2','Q9UJX3','Q9UJY1','Q9UM73','Q9UPZ9','Q9UQC2','Q9UQQ2','Q9Y281','Q9Y2K2','Q9Y618','Q9Y6I3','Q9Y6K9','Q9Y6W5','A1A5A8','A1L390','A4D111','A4D1P6','A4QN18','A6PVC2','A6PWT2','O00159','O00267','O00418','O00459','O00472','O00488','O00499','O00499-10','O00566','O00712','O00716','O14523','O14529','O14545','O14617','O14617-5','O14641','O14686','O14727','O14746','O14879','O14920','O14958','O14966','O14967','O14976','O15013','O15027','O15037','O15047','O15066','O15078','O15084','O15164','O15169','O15231','O15270','O15371','O15403','O15446','O15451','O15457','O15479','O43159','O43166','O43278','O43426','O43474','O43491','O43566','O43602','O43665','O43815','O43830','O60245','O60266','O60271','O60271-2','O60281','O60287','O60291','O60292','O60293','O60315','O60343','O60344','O60353','O60361','O60447','O60504','O60524','O60673','O60716','O60716-3','O60828','O75030-2','O75056','O75061','O75113','O75152','O75153','O75157','O75182','O75185','O75335','O75369','O75400','O75410','O75445','O75477','O75496','O75553','O75556','O75592','O75607','O75643','O75717','O75781','O75821','O75914-2','O75923','O75962','O75970-2','O94776','O94811','O94833','O94850','O94880','O94909','O94913','O94915','O94964','O94967','O94988','O94992','O95049','O95071','O95147','O95155','O95218','O95218-2','O95239','O95359','O95405-2','O95425','O95425-2','O95477','O95487','O95714','O95766','O95801','O95872','O96004','P00519-2','P00540','P01233','P01375','P01892','P01893','P02461','P02545','P02545-2','P02671','P04004','P04198','P05129','P06454','P06748-2','P06753','P07359','P07384','P07814','P07910','P07910-2','P07919','P08048','P08107','P08183','P08195','P08236','P08240','P10275','P10636','P11168','P11308','P11388-2','P11474','P11532','P11717','P11836','P12270','P12694','P12829','P12830','P13591','P13612','P13797','P13994','P14410','P14672','P14923','P15056','P15408','P15822','P15923-2','P15924','P15927','P16070','P16157','P16333','P16989','P17010','P17535','P17844','P18433','P18545','P18583-3','P18583-5','P18615','P18846','P18858','P18887','P19021','P19447','P20020','P20338','P20719','P20810','P21127-12','P21673','P22059','P22064','P22105','P23497','P24394','P24863','P24864','P25054','P25116','P25440','P26378','P27448-3','P27695','P27816','P28290','P28335','P28356','P29218','P29375','P29590','P30414','P30447','P30466','P30501','P31260','P31943','P32004','P32004-2','P32297','P33240','P33981-1','P34932','P34998','P35125','P35232','P35241','P35251','P35408','P35711','P35869','P36915','P37275','P40189','P40855','P41143','P41214','P41219','P41252','P41279','P41970','P42166','P42356','P42684','P42695','P42858','P43243','P43307','P43378','P45983','P46013','P46013-2','P46020','P46100','P46937','P46939','P47870','P48058','P48634-2','P48681','P48730-2','P48775','P49023-3','P49321','P49454','P49588','P49756','P49790','P49792','P49916','P49959','P50552','P50851','P51114-3','P51116','P51531','P51610','P51784','P51790','P51800','P51811','P51957','P52179','P52272-2','P52564','P52594','P52701','P53420','P53675','P53804','P53805','P53814-4','P53999','P54132','P54198','P54253','P54725','P55087','P55145','P55196','P55196-3','P55197','P55327','P55957','P56182','P56545','P57060','P57087','P57764','P58107','P58840','P59542','P60174','P62277','P62805','P62807','P63098','P78345','P78348','P78356','P81274','P81408','P82094','P83111','P98175','Q00341','Q00536','Q00587','Q00653','Q00722','Q00839','Q00839-2','Q00975','Q01151','Q01196','Q01484','Q01658','Q01804','Q01814','Q01970','Q02224','Q02241-2','Q02505','Q02539','Q02641','Q02750','Q03001','Q03060','Q03111','Q03135','Q03164','Q03188','Q03252','Q03468','Q03701','Q04637','Q04721','Q05209','Q05397','Q05586','Q05682-2','Q05682-3','Q06190','Q07157','Q07157-2','Q07866','Q07866-4','Q07912','Q08174','Q08174-2','Q08211','Q08289','Q08378','Q08499-12','Q08945','Q09019','Q09472','Q09666','Q0JRZ9','Q0VG06','Q0ZGT2','Q10571','Q12767-2','Q12769','Q12778','Q12792','Q12802','Q12830','Q12851','Q12857','Q12879','Q12906','Q12906-4','Q12906-5','Q12959','Q12959-2','Q12983','Q13002','Q13017','Q13023','Q13029','Q13067','Q13068','Q13085','Q13098','Q13111','Q13112','Q13123','Q13129','Q13164','Q13239','Q13255','Q13263','Q13330','Q13351','Q13409','Q13426','Q13428','Q13435','Q13439','Q13459','Q13509','Q13546','Q13563','Q13595','Q13596','Q13620','Q13671','Q13769','Q13796','Q13946','Q13972','Q14004','Q14004-2','Q14149','Q14156','Q14157-1','Q14160-2','Q14209','Q14241','Q14318','Q14444','Q14586','Q14669','Q14674','Q14676','Q14677','Q14684','Q14693','Q14789','Q14929','Q14980','Q15003','Q15021','Q15032','Q15032-2','Q15047','Q15118','Q15149-2','Q15149-4','Q15154','Q15170','Q15398','Q15424','Q15477','Q15554','Q15555','Q15561','Q15569','Q15637-2','Q15643','Q15651','Q15746','Q15825','Q15906','Q15911','Q16186','Q16512','Q16623','Q16626','Q16763','Q16816','Q16829','Q16881','Q16890','Q17RV1','Q1KMD3','Q1MSJ5','Q2KHR3','Q2VCE5','Q32MZ4','Q32NC0','Q3KQU3','Q3KR37','Q3MII6','Q3YEC7','Q4G0Z9','Q4W5G0','Q52WX2','Q53SF7','Q58A45','Q58A63','Q59HD5','Q5BJE1','Q5BKX5','Q5BKY9','Q5BKZ1','Q5CZA4','Q5CZH9','Q5H9R7','Q5JRA6','Q5JSH3','Q5JSZ5','Q5M775','Q5SVZ6','Q5T0D9','Q5T1V6','Q5T200','Q5T4C3','Q5T5P2','Q5T5U3','Q5T7V8','Q5T8D3','Q5T8I3','Q5T8P6-3','Q5T9S5','Q5TDH0','Q5TG08','Q5VT52','Q5VUA4','Q5VVJ2','Q5VWQ8','Q5VYS8','Q5VZ89','Q5VZL5','Q658Y4','Q66K43','Q676U5','Q68D20','Q68DE3','Q68DN1','Q69YH5','Q69YQ0','Q6BCY4','Q6DN12','Q6DSU6','Q6EMB2','Q6EMK4','Q6FIF0','Q6KC79','Q6KC79-2','Q6NUK4','Q6P0N0','Q6P0Q8','Q6P1N0','Q6P2H3','Q6P4R8','Q6P5Z2','Q6P6C2-2','Q6P995','Q6PCT2','Q6PGN9','Q6PGQ7','Q6PJ61','Q6PKG0','Q6PKG0-3','Q6PN88','Q6T4R5','Q6UB99','Q6UUV9','Q6UVJ0','Q6V1X1','Q6VMQ6','Q6W2J9','Q6WCQ1','Q6WKZ4','Q6XZF7','Q6Y7W6-3','Q6YL39','Q6ZMI0','Q6ZNB6','Q6ZNC4','Q6ZR49','Q6ZTQ6','Q6ZU33','Q6ZU80','Q6ZVG3','Q70CQ2','Q70E73','Q70EL4','Q70JA7','Q71RC2','Q71RE8','Q7KZ85','Q7KZI7-4','Q7L2J0','Q7LBC6','Q7RTP6','Q7Z3B3','Q7Z3D4','Q7Z3T8','Q7Z3V4','Q7Z434','Q7Z460','Q7Z460-3','Q7Z5L4','Q7Z620','Q7Z6B7','Q7Z6E9','Q7Z6J0','Q7Z6Z7','Q7Z7K6','Q86SJ2','Q86SQ0','Q86TC9','Q86TU1','Q86TV6','Q86U44','Q86UP2','Q86UR5','Q86UW7','Q86V48','Q86VI4','Q86VQ1','Q86W92-2','Q86XJ1','Q86XL3','Q86Y07','Q86YR5','Q86YS7','Q86YS7-2','Q86Z20','Q8IUD2','Q8IUI4','Q8IVL1','Q8IVT2','Q8IVT5','Q8IW35','Q8IWJ2','Q8IWK6','Q8IWR0','Q8IWS0-2','Q8IWW6','Q8IWX7','Q8IX01','Q8IX07','Q8IX15','Q8IXJ6','Q8IXZ2','Q8IY67-2','Q8IY81','Q8IYB3','Q8IYH5','Q8IZ21','Q8IZD4','Q8IZH2','Q8IZL8','Q8IZP2','Q8IZQ1','Q8N108','Q8N163','Q8N1Q3','Q8N2M8','Q8N350','Q8N3D4','Q8N3L3','Q8N3V7','Q8N3V7-2','Q8N3X1','Q8N3Z6','Q8N414','Q8N448','Q8N488','Q8N490','Q8N554','Q8N573','Q8N5A5','Q8N5C8','Q8N655','Q8N6H7','Q8N6L1','Q8N6T3','Q8N7K9','Q8N8S7','Q8N8S7-2','Q8N9M1','Q8N9T8','Q8NAE2','Q8NAJ6','Q8NBF6','Q8NBU5','Q8NC51-3','Q8NCN4','Q8ND56','Q8NDD1','Q8NDT2','Q8NE35','Q8NE35-2','Q8NEL9','Q8NEM2','Q8NET4','Q8NEY1','Q8NEY1-2','Q8NEZ4','Q8NFA0','Q8NFI4','Q8NFQ8','Q8NFT2','Q8NG31','Q8NHG8','Q8NHM5','Q8NHP6','Q8NI08','Q8NI27','Q8TAA9','Q8TBF4','Q8TBZ0','Q8TCN5','Q8TCU4','Q8TDJ6','Q8TEQ0','Q8TEQ6','Q8TEV9','Q8TEW0','Q8TF72','Q8WU79','Q8WUA7','Q8WUB8','Q8WUF5','Q8WUQ7','Q8WUR7','Q8WUX9','Q8WVM7','Q8WVT3','Q8WWI1','Q8WWI1-2','Q8WWI1-3','Q8WWQ0','Q8WXA3','Q8WXE0','Q8WXK8','Q8WYP5','Q8WYP5-2','Q8WZ64','Q92508','Q92541','Q92545','Q92576','Q92610','Q92613','Q92614-4','Q92615','Q92629','Q92738','Q92752','Q92764','Q92766','Q92777','Q92793','Q92841','Q92887','Q92888','Q92890','Q92908','Q92922','Q92925','Q93008','Q93052','Q93074','Q93075','Q969Q5','Q96AP0','Q96B23','Q96BV0','Q96BZ4','Q96C24','Q96C45','Q96D71','Q96DM3','Q96DT7','Q96E09','Q96ER3','Q96F63','Q96GS4','Q96GX8','Q96GY3','Q96I24','Q96IY1','Q96JB1','Q96JM2','Q96JM3','Q96JN0','Q96K37','Q96K76','Q96KQ7','Q96KQ7-2','Q96KW2','Q96L73','Q96L91','Q96LW7','Q96MF7','Q96MT7','Q96MW7','Q96N67-2','Q96NB3','Q96P48','Q96P48-6','Q96PE2','Q96PK2','Q96PY5','Q96Q15','Q96QE2','Q96RG2','Q96RL7','Q96RT1','Q96RU3','Q96RV3','Q96RY5','Q96SB3','Q96SB4','Q96SB8','Q96ST3','Q96T23','Q96T25','Q96TA1','Q99549','Q99567','Q99569','Q99572','Q99590','Q99615','Q99626','Q99638','Q99650','Q99661','Q99684','Q99700','Q99704','Q99717','Q99747','Q99871','Q99873','Q99952','Q99956','Q99962','Q99966','Q99986','Q99996','Q9BQ67','Q9BQ70','Q9BQA9','Q9BQL6','Q9BRD0','Q9BRG2','Q9BRH9','Q9BRN9','Q9BSE2','Q9BTA9','Q9BTC0','Q9BTX1','Q9BU19','Q9BVC5','Q9BVI0','Q9BVS4','Q9BW34','Q9BW85','Q9BX66','Q9BX66-3','Q9BXF6','Q9BXK5','Q9BXS6','Q9BY15','Q9BY89','Q9BYJ9','Q9BYX2','Q9BZB8','Q9BZC7','Q9BZQ8','Q9BZV1','Q9C026-5','Q9C0C2','Q9C0E8','Q9C0G0','Q9GZS1','Q9GZV8','Q9GZZ9','Q9H0E9-2','Q9H0H3','Q9H1B7','Q9H1K1','Q9H2D6','Q9H2P0','Q9H2U1','Q9H2Y7','Q9H3P2','Q9H4X1','Q9H5I1','Q9H5U6','Q9H6S0','Q9H6Y2','Q9H728','Q9H773','Q9H7D0','Q9H7E2','Q9H7N4','Q9H814','Q9H822','Q9H910','Q9H956','Q9H9B1','Q9H9J4','Q9HAH7','Q9HAS0','Q9HAW4','Q9HBG4','Q9HBL0','Q9HBL6','Q9HBZ2','Q9HC35','Q9HCD5','Q9HCE1','Q9HCF4','Q9HCK8','Q9HCU4','Q9HD67','Q9NPA5','Q9NPQ8','Q9NPR2','Q9NQ55','Q9NQA5','Q9NQC7','Q9NQV6','Q9NQW6','Q9NR09','Q9NRA0','Q9NRA8','Q9NRF8','Q9NRF9','Q9NS56','Q9NS85','Q9NSC7','Q9NSI6-2','Q9NSU2','Q9NSV0','Q9NUD5','Q9NUQ3','Q9NUW8','Q9NVP4','Q9NVT9','Q9NVV0','Q9NXL9','Q9NXR1-2','Q9NXZ1','Q9NY61','Q9NYF8','Q9NYU2','Q9NYV4','Q9NYZ3','Q9NZ38','Q9NZ56','Q9NZC7','Q9NZJ5','Q9NZL4','Q9NZM1','Q9NZM3','Q9NZM5','Q9NZN8','Q9NZV8','Q9P0P8','Q9P1T7','Q9P1Y5','Q9P207','Q9P227','Q9P242','Q9P275','Q9P2D1','Q9P2F8','Q9P2G1','Q9P2G4','Q9P2H5','Q9P2N5','Q9P2Z0','Q9UBD8','Q9UBF8-2','Q9UBL6','Q9UDY4','Q9UER7','Q9UGR2','Q9UGU0','Q9UGY1','Q9UHB7','Q9UHW9','Q9UI08','Q9UIF8','Q9UIF9','Q9UIG0','Q9UIU6','Q9UJ41','Q9UJQ4','Q9UJX4','Q9UKF7','Q9UKI8-2','Q9UKJ3','Q9UKL0','Q9UKS7','Q9UKV0-7','Q9UKV3','Q9UKV5','Q9UKY1','Q9UL51','Q9UL60','Q9ULE0','Q9ULH7','Q9ULI4','Q9ULJ3','Q9ULJ7','Q9ULJ8','Q9ULL5','Q9ULT8','Q9ULV0','Q9ULW0','Q9UM00','Q9UMQ3','Q9UMR2','Q9UN30','Q9UN79','Q9UN86','Q9UN86-2','Q9UNE0','Q9UNN4','Q9UNX4','Q9UNZ2','Q9UPN4','Q9UPN7','Q9UPQ0','Q9UPQ6','Q9UPS8','Q9UPU5','Q9UPX8','Q9UPY3','Q9UQ35','Q9UQ88','Q9UQR0','Q9Y2H0-2','Q9Y2H5','Q9Y2H6','Q9Y2K1','Q9Y2K3','Q9Y2L6','Q9Y2N3','Q9Y2R4','Q9Y2W1','Q9Y314','Q9Y3F4','Q9Y3Q8','Q9Y3T9','Q9Y450','Q9Y462','Q9Y496','Q9Y4A5','Q9Y4B5','Q9Y4F3','Q9Y4K1','Q9Y580','Q9Y5P4','Q9Y608','Q9Y653','Q9Y679','Q9Y6D5','Q9Y6D6','Q9Y6D9','Q9Y6G9','Q9Y6K1','Q9Y6M4','Q9Y6M4-3','Q9Y6M7','Q9Y6N7']

In [ ]:
### RETRIEVE GENE NAMES FROM UNIPROT

# Use UniProt's recommended Python 3 script (available at https://www.uniprot.org/help/api_idmapping)

# Create partial URL

url = "https://www.uniprot.org/uploadlists/"

# Define parameters

params = {
    
"from" : "ID", # Assume protein names are in format "ID"
"to" : "GENENAME", # Retrieve IDs in "GENENAME" format
"format" : "tab", # Produce tab-delimited output
"query" : "", # The query protein ID will be defined during the loop
    
}

# Create an empty list to store the results in

results = []

for i in substrates: 
    params[ "query" ] = str( i ) # Enter the substrate ID
    data = urllib.parse.urlencode( params ) 
    data = data.encode( "utf-8" )
    req = urllib.request.Request( url, data ) # Run query in Uniprot
    with urllib.request.urlopen( req ) as f:
        response = f.read()
    line = response.decode( "utf-8" ) 
    results.append( line ) # Store results

In [ ]:
# Split the results into a list of lists without "\n" characters

results2 = []

for n, i in enumerate( results ):
    results2.append( i.split() )

If the above UniProt-querying loop fails to run, un-comment and run the cell below for a snapshot of the data retrieved on 27 Jan 2020

In [ ]:
# results2 = [['From', 'To', 'O00141', 'SGK1'],['From', 'To', 'O00161', 'SNAP23'],['From', 'To', 'O00257', 'CBX4'],['From', 'To', 'O14745', 'SLC9A3R1'],['From', 'To', 'O14757', 'CHEK1'],['From', 'To', 'O14964', 'HGS'],['From', 'To', 'O14994', 'SYN3'],['From', 'To', 'O15273', 'TCAP'],['From', 'To', 'O15357', 'INPPL1'],['From', 'To', 'O15492', 'RGS16'],['From', 'To', 'O15530', 'PDPK1'],['From', 'To', 'O43521', 'BCL2L11'],['From', 'To', 'O43586', 'PSTPIP1'],['From', 'To', 'O43914', 'TYROBP'],['From', 'To', 'O60381', 'HBP1'],['From', 'To', 'O60506', 'SYNCRIP'],['From', 'To', 'O60674', 'JAK2'],['From', 'To', 'O75582', 'RPS6KA5'],['From', 'To', 'O75676', 'RPS6KA4'],['From', 'To', 'O75914', 'PAK3'],['From', 'To', 'O95997', 'PTTG1'],['From', 'To', 'O96017', 'CHEK2'],['From', 'To', 'P00519', 'ABL1'],['From', 'To', 'P00533', 'EGFR'],['From', 'To', 'P01100', 'FOS'],['From', 'To', 'P01236', 'PRL'],['From', 'To', 'P01350', 'GAST'],['From', 'To', 'P02686', 'MBP'],['From', 'To', 'P03372', 'ESR1'],['From', 'To', 'P04049', 'RAF1'],['From', 'To', 'P04150', 'NR3C1'],['From', 'To', 'P04629', 'NTRK1'],['From', 'To', 'P04637', 'TP53'],['From', 'To', 'P05106', 'ITGB3'],['From', 'To', 'P05198', 'EIF2S1'],['From', 'To', 'P05412', 'JUN'],['From', 'To', 'P05787', 'KRT8'],['From', 'To', 'P06213', 'INSR'],['From', 'To', 'P06239', 'LCK'],['From', 'To', 'P06401', 'PGR'],['From', 'To', 'P06733', 'ENO1'],['From', 'To', 'P07101', 'TH'],['From', 'To', 'P07203', 'GPX1'],['From', 'To', 'P07550', 'ADRB2'],['From', 'To', 'P07947', 'YES1'],['From', 'To', 'P07948', 'LYN'],['From', 'To', 'P08047', 'SP1'],['From', 'To', 'P08069', 'IGF1R'],['From', 'To', 'P08100', 'RHO'],['From', 'To', 'P08631', 'HCK'],['From', 'To', 'P08670', 'VIM'],['From', 'To', 'P09038', 'FGF2'],['From', 'To', 'P09619', 'PDGFRB'],['From', 'To', 'P09769', 'FGR'],['From', 'To', 'P0C0S8', 'H2AC11', 'P0C0S8', 'H2AC13', 'P0C0S8', 'H2AC15', 'P0C0S8', 'H2AC16', 'P0C0S8', 'H2AC17'],['From', 'To', 'P10242', 'MYB'],['From', 'To', 'P10244', 'MYBL2'],['From', 'To', 'P10721', 'KIT'],['From', 'To', 'P10747', 'CD28'],['From', 'To', 'P10828', 'THRB'],['From', 'To', 'P10912', 'GHR'],['From', 'To', 'P11137', 'MAP2'],['From', 'To', 'P11171', 'EPB41'],['From', 'To', 'P11274', 'BCR'],['From', 'To', 'P11362', 'FGFR1'],['From', 'To', 'P11388', 'TOP2A'],['From', 'To', 'P11831', 'SRF'],['From', 'To', 'P11912', 'CD79A'],['From', 'To', 'P12272', 'PTHLH'],['From', 'To', 'P12318', 'FCGR2A'],['From', 'To', 'P12931', 'SRC'],['From', 'To', 'P13569', 'CFTR'],['From', 'To', 'P13693', 'TPT1'],['From', 'To', 'P14317', 'HCLS1'],['From', 'To', 'P14598', 'NCF1'],['From', 'To', 'P14859', 'POU2F1'],['From', 'To', 'P15172', 'MYOD1'],['From', 'To', 'P15260', 'IFNGR1'],['From', 'To', 'P15336', 'ATF2'],['From', 'To', 'P15941', 'MUC1'],['From', 'To', 'P16104', 'H2AFX'],['From', 'To', 'P16220', 'CREB1'],['From', 'To', 'P16234', 'PDGFRA'],['From', 'To', 'P16410', 'CTLA4'],['From', 'To', 'P16885', 'PLCG2'],['From', 'To', 'P17252', 'PRKCA'],['From', 'To', 'P17275', 'JUNB'],['From', 'To', 'P17480', 'UBTF'],['From', 'To', 'P17542', 'TAL1'],['From', 'To', 'P17600', 'SYN1'],['From', 'To', 'P17655', 'CAPN2'],['From', 'To', 'P17676', 'CEBPB'],['From', 'To', 'P18031', 'PTPN1'],['From', 'To', 'P18206', 'VCL'],['From', 'To', 'P18507', 'GABRG2'],['From', 'To', 'P19105', 'MYL12A'],['From', 'To', 'P19174', 'PLCG1'],['From', 'To', 'P19235', 'EPOR'],['From', 'To', 'P19419', 'ELK1'],['From', 'To', 'P19429', 'TNNI3'],['From', 'To', 'P19793', 'RXRA'],['From', 'To', 'P20138', 'CD33'],['From', 'To', 'P20700', 'LMNB1'],['From', 'To', 'P20936', 'RASA1'],['From', 'To', 'P20963', 'CD247'],['From', 'To', 'P22001', 'KCNA3'],['From', 'To', 'P22314', 'UBA1'],['From', 'To', 'P22607', 'FGFR3'],['From', 'To', 'P22681', 'CBL'],['From', 'To', 'P23396', 'RPS3'],['From', 'To', 'P23443', 'RPS6KB1'],['From', 'To', 'P23458', 'JAK1'],['From', 'To', 'P23528', 'CFL1'],['From', 'To', 'P23634', 'ATP2B4'],['From', 'To', 'P24844', 'MYL9'],['From', 'To', 'P24928', 'POLR2A'],['From', 'To', 'P25963', 'NFKBIA'],['From', 'To', 'P27448', 'MARK3'],['From', 'To', 'P27708', 'CAD'],['From', 'To', 'P27986', 'PIK3R1'],['From', 'To', 'P28562', 'DUSP1'],['From', 'To', 'P28749', 'RBL1'],['From', 'To', 'P29320', 'EPHA3'],['From', 'To', 'P29322', 'EPHA8'],['From', 'To', 'P29350', 'PTPN6'],['From', 'To', 'P29353', 'SHC1'],['From', 'To', 'P29597', 'TYK2'],['From', 'To', 'P30260', 'CDC27'],['From', 'To', 'P30304', 'CDC25A'],['From', 'To', 'P30305', 'CDC25B'],['From', 'To', 'P30307', 'CDC25C'],['From', 'To', 'P30419', 'NMT1'],['From', 'To', 'P30443', 'HLA-A'],['From', 'To', 'P30530', 'AXL'],['From', 'To', 'P31152', 'MAPK4'],['From', 'To', 'P31946', 'YWHAB'],['From', 'To', 'P33241', 'LSP1'],['From', 'To', 'P33778', 'HIST1H2BB'],['From', 'To', 'P33981', 'TTK'],['From', 'To', 'P33991', 'MCM4'],['From', 'To', 'P35222', 'CTNNB1'],['From', 'To', 'P35568', 'IRS1'],['From', 'To', 'P35611', 'ADD1'],['From', 'To', 'P35612', 'ADD2'],['From', 'To', 'P35637', 'FUS'],['From', 'To', 'P36888', 'FLT3'],['From', 'To', 'P36956', 'SREBF1'],['From', 'To', 'P38398', 'BRCA1'],['From', 'To', 'P38432', 'COIL'],['From', 'To', 'P38936', 'CDKN1A'],['From', 'To', 'P39880', 'CUX1'],['From', 'To', 'P40259', 'CD79B'],['From', 'To', 'P41212', 'ETV6'],['From', 'To', 'P41743', 'PRKCI'],['From', 'To', 'P42229', 'STAT5A'],['From', 'To', 'P42702', 'LIFR'],['From', 'To', 'P42768', 'WAS'],['From', 'To', 'P43354', 'NR4A2'],['From', 'To', 'P46527', 'CDKN1B'],['From', 'To', 'P46695', 'IER3'],['From', 'To', 'P48751', 'SLC4A3'],['From', 'To', 'P49023', 'PXN'],['From', 'To', 'P49407', 'ARRB1'],['From', 'To', 'P49715', 'CEBPA'],['From', 'To', 'P49768', 'PSEN1'],['From', 'To', 'P49841', 'GSK3B'],['From', 'To', 'P49917', 'LIG4'],['From', 'To', 'P50549', 'ETV1'],['From', 'To', 'P50613', 'CDK7'],['From', 'To', 'P51674', 'GPM6A'],['From', 'To', 'P51692', 'STAT5B'],['From', 'To', 'P51812', 'RPS6KA3'],['From', 'To', 'P51813', 'BMX'],['From', 'To', 'P52735', 'VAV2'],['From', 'To', 'P52799', 'EFNB2'],['From', 'To', 'P52926', 'HMGA2'],['From', 'To', 'P53355', 'DAPK1'],['From', 'To', 'P53667', 'LIMK1'],['From', 'To', 'P53671', 'LIMK2'],['From', 'To', 'P53778', 'MAPK12'],['From', 'To', 'P54259', 'ATN1'],['From', 'To', 'P55211', 'CASP9'],['From', 'To', 'P56270', 'MAZ'],['From', 'To', 'P56945', 'BCAR1'],['From', 'To', 'P58012', 'FOXL2'],['From', 'To', 'P60953', 'CDC42'],['From', 'To', 'P61925', 'PKIA'],['From', 'To', 'P61978', 'HNRNPK'],['From', 'To', 'P62158', 'CALM1', 'P62158', 'CALM2', 'P62158', 'CALM3'],['From', 'To', 'P62993', 'GRB2'],['From', 'To', 'P68400', 'CSNK2A1'],['From', 'To', 'P68431', 'H3C1', 'P68431', 'H3C10', 'P68431', 'H3C11', 'P68431', 'H3C12', 'P68431', 'H3C2', 'P68431', 'H3C3', 'P68431', 'H3C4', 'P68431', 'H3C6', 'P68431', 'H3C7', 'P68431', 'H3C8'],['From', 'To', 'P78314', 'SH3BP2'],['From', 'To', 'P78347', 'GTF2I'],['From', 'To', 'P78527', 'PRKDC'],['From', 'To', 'P78536', 'ADAM17'],['From', 'To', 'P84022', 'SMAD3'],['From', 'To', 'P84243', 'H3-3A', 'P84243', 'H3-3B'],['From', 'To', 'Q00169', 'PITPNA'],['From', 'To', 'Q00535', 'CDK5'],['From', 'To', 'Q01892', 'SPIB'],['From', 'To', 'Q02078', 'MEF2A'],['From', 'To', 'Q02241', 'KIF23'],['From', 'To', 'Q02363', 'ID2'],['From', 'To', 'Q02763', 'TEK'],['From', 'To', 'Q03721', 'KCNC4'],['From', 'To', 'Q04206', 'RELA'],['From', 'To', 'Q04912', 'MST1R'],['From', 'To', 'Q05513', 'PRKCZ'],['From', 'To', 'Q05655', 'PRKCD'],['From', 'To', 'Q05682', 'CALD1'],['From', 'To', 'Q06124', 'PTPN11'],['From', 'To', 'Q06413', 'MEF2C'],['From', 'To', 'Q07666', 'KHDRBS1'],['From', 'To', 'Q07820', 'MCL1'],['From', 'To', 'Q08379', 'GOLGA2'],['From', 'To', 'Q12772', 'SREBF2'],['From', 'To', 'Q12888', 'TP53BP1'],['From', 'To', 'Q12972', 'PPP1R8'],['From', 'To', 'Q13094', 'LCP2'],['From', 'To', 'Q13131', 'PRKAA1'],['From', 'To', 'Q13153', 'PAK1'],['From', 'To', 'Q13158', 'FADD'],['From', 'To', 'Q13177', 'PAK2'],['From', 'To', 'Q13224', 'GRIN2B'],['From', 'To', 'Q13322', 'GRB10'],['From', 'To', 'Q13415', 'ORC1'],['From', 'To', 'Q13418', 'ILK'],['From', 'To', 'Q13424', 'SNTA1'],['From', 'To', 'Q13444', 'ADAM15'],['From', 'To', 'Q13480', 'GAB1'],['From', 'To', 'Q13485', 'SMAD4'],['From', 'To', 'Q13507', 'TRPC3'],['From', 'To', 'Q13541', 'EIF4EBP1'],['From', 'To', 'Q13651', 'IL10RA'],['From', 'To', 'Q13765', 'NACA'],['From', 'To', 'Q13813', 'SPTAN1'],['From', 'To', 'Q14005', 'IL16'],['From', 'To', 'Q14118', 'DAG1'],['From', 'To', 'Q14191', 'WRN'],['From', 'To', 'Q14247', 'CTTN'],['From', 'To', 'Q14289', 'PTK2B'],['From', 'To', 'Q14847', 'LASP1'],['From', 'To', 'Q14934', 'NFATC4'],['From', 'To', 'Q15139', 'PRKD1'],['From', 'To', 'Q15149', 'PLEC'],['From', 'To', 'Q15418', 'RPS6KA1'],['From', 'To', 'Q15648', 'MED1'],['From', 'To', 'Q15653', 'NFKBIB'],['From', 'To', 'Q15788', 'NCOA1'],['From', 'To', 'Q15796', 'SMAD2'],['From', 'To', 'Q15831', 'STK11'],['From', 'To', 'Q16760', 'DGKD'],['From', 'To', 'Q16821', 'PPP1R3A'],['From', 'To', 'Q16828', 'DUSP6'],['From', 'To', 'Q7KZI7', 'MARK2'],['From', 'To', 'Q86WB0', 'ZC3HC1'],['From', 'To', 'Q86WV1', 'SKAP1'],['From', 'To', 'Q8TDC3', 'BRSK1'],['From', 'To', 'Q8WXE1', 'ATRIP'],['From', 'To', 'Q92731', 'ESR2'],['From', 'To', 'Q92918', 'MAP4K1'],['From', 'To', 'Q92934', 'BAD'],['From', 'To', 'Q92974', 'ARHGEF2'],['From', 'To', 'Q99683', 'MAP3K5'],['From', 'To', 'Q9BUB5', 'MKNK1'],['From', 'To', 'Q9BXS5', 'AP1M1'],['From', 'To', 'Q9BXW9', 'FANCD2'],['From', 'To', 'Q9H0H5', 'RACGAP1'],['From', 'To', 'Q9H1A4', 'ANAPC1'],['From', 'To', 'Q9H1D0', 'TRPV6'],['From', 'To', 'Q9H8V3', 'ECT2'],['From', 'To', 'Q9HBA0', 'TRPV4'],['From', 'To', 'Q9NQ66', 'PLCB1'],['From', 'To', 'Q9NQS7', 'INCENP'],['From', 'To', 'Q9NQU5', 'PAK6'],['From', 'To', 'Q9NRQ2', 'PLSCR4'],['From', 'To', 'Q9NRY4', 'ARHGAP35'],['From', 'To', 'Q9NXH3', 'PPP1R14D'],['From', 'To', 'Q9NYV6', 'RRN3'],['From', 'To', 'Q9UBS0', 'RPS6KB2'],['From', 'To', 'Q9UEW8', 'STK39'],['From', 'To', 'Q9UJX2', 'CDC23'],['From', 'To', 'Q9UJX3', 'ANAPC7'],['From', 'To', 'Q9UJY1', 'HSPB8'],['From', 'To', 'Q9UM73', 'ALK'],['From', 'To', 'Q9UPZ9','ICK'],['From', 'To', 'Q9UQC2', 'GAB2'],['From', 'To', 'Q9UQQ2', 'SH2B3'],['From', 'To', 'Q9Y281', 'CFL2'],['From', 'To', 'Q9Y2K2', 'SIK3'],['From', 'To', 'Q9Y618', 'NCOR2'],['From', 'To', 'Q9Y6I3', 'EPN1'],['From', 'To', 'Q9Y6K9', 'IKBKG'],['From', 'To', 'Q9Y6W5', 'WASF2'],['From', 'To', 'A1A5A8', 'SIK3'],['From', 'To', 'A1L390', 'PLEKHG3'],['From', 'To', 'A4D111', 'LOC401308'],['From', 'To', 'A4D1P6', 'WDR91'],['From', 'To', 'A4QN18', 'SEC16A'],['From', 'To', 'A6PVC2', 'TTLL8'],['From', 'To'],['From', 'To', 'O00159', 'MYO1C'],['From', 'To', 'O00267', 'SUPT5H'],['From', 'To', 'O00418', 'EEF2K'],['From', 'To', 'O00459', 'PIK3R2'],['From', 'To', 'O00472', 'ELL2'],['From', 'To', 'O00488', 'ZNF593'],['From', 'To', 'O00499', 'BIN1'],['From', 'To'],['From', 'To', 'O00566', 'MPHOSPH10'],['From', 'To', 'O00712', 'NFIB'],['From', 'To', 'O00716', 'E2F3'],['From', 'To', 'O14523', 'C2CD2L'],['From', 'To', 'O14529', 'CUX2'],['From', 'To', 'O14545', 'TRAFD1'],['From', 'To', 'O14617', 'AP3D1'],['From', 'To'],['From', 'To', 'O14641', 'DVL2'],['From', 'To', 'O14686', 'KMT2D'],['From', 'To', 'O14727', 'APAF1'],['From', 'To', 'O14746', 'TERT'],['From', 'To', 'O14879', 'IFIT3'],['From', 'To', 'O14920', 'IKBKB'],['From', 'To', 'O14958', 'CASQ2'],['From', 'To', 'O14966', 'RAB29'],['From', 'To', 'O14967', 'CLGN'],['From', 'To', 'O14976', 'GAK'],['From', 'To', 'O15013', 'ARHGEF10'],['From', 'To', 'O15027', 'SEC16A'],['From', 'To', 'O15037', 'KHNYN'],['From', 'To', 'O15047', 'SETD1A'],['From', 'To', 'O15066', 'KIF3B'],['From', 'To', 'O15078', 'CEP290'],['From', 'To', 'O15084', 'ANKRD28'],['From', 'To', 'O15164', 'TRIM24'],['From', 'To', 'O15169', 'AXIN1'],['From', 'To', 'O15231', 'ZNF185'],['From', 'To', 'O15270', 'SPTLC2'],['From', 'To', 'O15371', 'EIF3D'],['From', 'To', 'O15403', 'SLC16A6'],['From', 'To', 'O15446', 'CD3EAP'],['From', 'To'],['From', 'To', 'O15457', 'MSH4'],['From', 'To', 'O15479', 'MAGEB2'],['From', 'To', 'O43159', 'RRP8'],['From', 'To', 'O43166', 'SIPA1L1'],['From', 'To', 'O43278', 'SPINT1'],['From', 'To', 'O43426', 'SYNJ1'],['From', 'To', 'O43474', 'KLF4'],['From', 'To', 'O43491', 'EPB41L2'],['From', 'To', 'O43566', 'RGS14'],['From', 'To', 'O43602', 'DCX'],['From', 'To', 'O43665', 'RGS10'],['From', 'To', 'O43815', 'STRN'],['From', 'To'],['From', 'To', 'O60245', 'PCDH7'],['From', 'To', 'O60266', 'ADCY3'],['From', 'To', 'O60271', 'SPAG9'],['From', 'To'],['From', 'To', 'O60281', 'ZNF292'],['From', 'To', 'O60287', 'URB1'],['From', 'To', 'O60291', 'MGRN1'],['From', 'To', 'O60292', 'SIPA1L3'],['From', 'To', 'O60293', 'ZFC3H1'],['From', 'To', 'O60315', 'ZEB2'],['From', 'To', 'O60343', 'TBC1D4'],['From', 'To', 'O60344', 'ECE2', 'O60344', 'EEF1AKMT4', 'O60344', 'EEF1AKMT4-ECE2'],['From', 'To', 'O60353', 'FZD6'],['From', 'To', 'O60361', 'NME2P1'],['From', 'To', 'O60447', 'EVI5'],['From', 'To', 'O60504', 'SORBS3'],['From', 'To', 'O60524', 'NEMF'],['From', 'To', 'O60673', 'REV3L'],['From', 'To', 'O60716', 'CTNND1'],['From', 'To'],['From', 'To', 'O60828', 'PQBP1'],['From', 'To'],['From', 'To', 'O75056', 'SDC3'],['From', 'To', 'O75061', 'DNAJC6'],['From', 'To', 'O75113', 'N4BP1'],['From', 'To', 'O75152', 'ZC3H11A'],['From', 'To', 'O75153', 'CLUH'],['From', 'To', 'O75157', 'TSC22D2'],['From', 'To', 'O75182', 'SIN3B'],['From', 'To', 'O75185', 'ATP2C2'],['From', 'To', 'O75335', 'PPFIA4'],['From', 'To', 'O75369', 'FLNB'],['From', 'To', 'O75400', 'PRPF40A'],['From', 'To', 'O75410', 'TACC1'],['From', 'To', 'O75445', 'USH2A'],['From', 'To', 'O75477', 'ERLIN1'],['From', 'To', 'O75496', 'GMNN'],['From', 'To', 'O75553', 'DAB1'],['From', 'To', 'O75556', 'SCGB2A1'],['From', 'To', 'O75592', 'MYCBP2'],['From', 'To', 'O75607', 'NPM3'],['From', 'To', 'O75643', 'SNRNP200'],['From', 'To', 'O75717', 'WDHD1'],['From', 'To', 'O75781', 'PALM'],['From', 'To', 'O75821', 'EIF3G'],['From', 'To'],['From', 'To', 'O75923', 'DYSF'],['From', 'To', 'O75962', 'TRIO'],['From', 'To'],['From', 'To', 'O94776', 'MTA2'],['From', 'To', 'O94811', 'TPPP'],['From', 'To', 'O94833', 'DST'],['From', 'To', 'O94850', 'DDN'],['From', 'To', 'O94880', 'PHF14'],['From', 'To', 'O94909', 'MICAL3'],['From', 'To', 'O94913', 'PCF11'],['From', 'To', 'O94915', 'FRYL'],['From', 'To', 'O94964', 'SOGA1'],['From', 'To', 'O94967', 'WDR47'],['From', 'To', 'O94988', 'FAM13A'],['From', 'To', 'O94992', 'HEXIM1'],['From', 'To', 'O95049', 'TJP3'],['From', 'To', 'O95071', 'UBR5'],['From', 'To', 'O95147', 'DUSP14'],['From', 'To', 'O95155', 'UBE4B'],['From', 'To', 'O95218', 'ZRANB2'],['From', 'To'],['From', 'To', 'O95239', 'KIF4A'],['From', 'To', 'O95359', 'TACC2'],['From', 'To'],['From', 'To', 'O95425', 'SVIL'],['From', 'To'],['From', 'To', 'O95477', 'ABCA1'],['From', 'To', 'O95487', 'SEC24B'],['From', 'To', 'O95714', 'HERC2'],['From', 'To', 'O95766', 'CCZ1', 'O95766', 'CCZ1B'],['From', 'To', 'O95801', 'TTC4'],['From', 'To', 'O95872', 'GPANK1'],['From', 'To', 'O96004', 'HAND1'],['From', 'To'],['From', 'To', 'P00540', 'MOS'],['From', 'To', 'P01233', 'CGB3', 'P01233', 'CGB5', 'P01233', 'CGB7', 'P01233', 'CGB8'],['From', 'To', 'P01375', 'TNF'],['From', 'To', 'P01892', 'HLA-A'],['From', 'To', 'P01893', 'HLA-H'],['From', 'To', 'P02461', 'COL3A1'],['From', 'To', 'P02545', 'LMNA'],['From', 'To'],['From', 'To', 'P02671', 'FGA'],['From', 'To', 'P04004', 'VTN'],['From', 'To', 'P04198', 'MYCN'],['From', 'To', 'P05129', 'PRKCG'],['From', 'To', 'P06454', 'PTMA'],['From', 'To'],['From', 'To', 'P06753', 'TPM3'],['From', 'To', 'P07359', 'GP1BA'],['From', 'To', 'P07384', 'CAPN1'],['From', 'To', 'P07814', 'EPRS'],['From', 'To', 'P07910', 'HNRNPC'],['From', 'To'],['From', 'To', 'P07919', 'UQCRH'],['From', 'To', 'P08048', 'ZFY'],['From', 'To', 'P08107', 'HSPA1A', 'P08107', 'HSPA1B'],['From', 'To', 'P08183', 'ABCB1'],['From', 'To', 'P08195', 'SLC3A2'],['From', 'To', 'P08236', 'GUSB'],['From', 'To', 'P08240', 'SRPRA'],['From', 'To', 'P10275', 'AR'],['From', 'To', 'P10636', 'MAPT'],['From', 'To', 'P11168', 'SLC2A2'],['From', 'To', 'P11308', 'ERG'],['From', 'To'],['From', 'To', 'P11474', 'ESRRA'],['From', 'To', 'P11532', 'DMD'],['From', 'To', 'P11717', 'IGF2R'],['From', 'To', 'P11836', 'MS4A1'],['From', 'To', 'P12270', 'TPR'],['From', 'To', 'P12694', 'BCKDHA'],['From', 'To', 'P12829', 'MYL4'],['From', 'To', 'P12830', 'CDH1'],['From', 'To', 'P13591', 'NCAM1'],['From', 'To', 'P13612', 'ITGA4'],['From', 'To', 'P13797', 'PLS3'],['From', 'To', 'P13994', 'CCDC130'],['From', 'To', 'P14410', 'SI'],['From', 'To', 'P14672', 'SLC2A4'],['From', 'To', 'P14923', 'JUP'],['From', 'To', 'P15056', 'BRAF'],['From', 'To', 'P15408', 'FOSL2'],['From', 'To', 'P15822', 'HIVEP1'],['From', 'To'],['From', 'To', 'P15924', 'DSP'],['From', 'To', 'P15927', 'RPA2'],['From', 'To', 'P16070', 'CD44'],['From', 'To', 'P16157', 'ANK1'],['From', 'To', 'P16333', 'NCK1'],['From', 'To', 'P16989', 'YBX3'],['From', 'To', 'P17010', 'ZFX'],['From', 'To', 'P17535', 'JUND'],['From', 'To', 'P17844', 'DDX5'],['From', 'To', 'P18433', 'PTPRA'],['From', 'To', 'P18545', 'PDE6G'],['From', 'To'],['From', 'To'],['From', 'To', 'P18615', 'NELFE'],['From', 'To', 'P18846', 'ATF1'],['From', 'To', 'P18858', 'LIG1'],['From', 'To', 'P18887', 'XRCC1'],['From', 'To', 'P19021', 'PAM'],['From', 'To', 'P19447', 'ERCC3'],['From', 'To', 'P20020', 'ATP2B1'],['From', 'To', 'P20338', 'RAB4A'],['From', 'To', 'P20719', 'HOXA5'],['From', 'To', 'P20810', 'CAST'],['From', 'To'],['From', 'To', 'P21673', 'SAT1'],['From', 'To', 'P22059', 'OSBP'],['From', 'To', 'P22064', 'LTBP1'],['From', 'To', 'P22105', 'TNXB'],['From', 'To', 'P23497', 'SP100'],['From', 'To', 'P24394', 'IL4R'],['From', 'To', 'P24863', 'CCNC'],['From', 'To', 'P24864', 'CCNE1'],['From', 'To', 'P25054', 'APC'],['From', 'To', 'P25116', 'F2R'],['From', 'To', 'P25440', 'BRD2'],['From', 'To', 'P26378', 'ELAVL4'],['From', 'To'],['From', 'To', 'P27695', 'APEX1'],['From', 'To', 'P27816', 'MAP4'],['From', 'To', 'P28290', 'ITPRID2'],['From', 'To', 'P28335', 'HTR2C'],['From', 'To', 'P28356', 'HOXD9'],['From', 'To', 'P29218', 'IMPA1'],['From', 'To', 'P29375', 'KDM5A'],['From', 'To', 'P29590', 'PML'],['From', 'To', 'P30414', 'NKTR'],['From', 'To', 'P30447', 'HLA-A'],['From', 'To', 'P30466', 'HLA-B'],['From', 'To', 'P30501', 'HLA-C'],['From', 'To', 'P31260', 'HOXA10'],['From', 'To', 'P31943', 'HNRNPH1'],['From', 'To', 'P32004', 'L1CAM'],['From', 'To'],['From', 'To', 'P32297', 'CHRNA3'],['From', 'To', 'P33240', 'CSTF2'],['From', 'To'],['From', 'To', 'P34932', 'HSPA4'],['From', 'To', 'P34998', 'CRHR1'],['From', 'To', 'P35125', 'USP6'],['From', 'To', 'P35232', 'PHB'],['From', 'To', 'P35241', 'RDX'],['From', 'To', 'P35251', 'RFC1'],['From', 'To', 'P35408', 'PTGER4'],['From', 'To', 'P35711', 'SOX5'],['From', 'To', 'P35869', 'AHR'],['From', 'To', 'P36915', 'GNL1'],['From', 'To', 'P37275', 'ZEB1'],['From', 'To', 'P40189', 'IL6ST'],['From', 'To', 'P40855', 'PEX19'],['From', 'To', 'P41143', 'OPRD1'],['From', 'To', 'P41214', 'EIF2D'],['From', 'To', 'P41219', 'PRPH'],['From', 'To', 'P41252', 'IARS'],['From', 'To', 'P41279', 'MAP3K8'],['From', 'To', 'P41970', 'ELK3'],['From', 'To', 'P42166', 'TMPO'],['From', 'To', 'P42356', 'PI4KA'],['From', 'To', 'P42684', 'ABL2'],['From', 'To', 'P42695', 'NCAPD3'],['From', 'To', 'P42858', 'HTT'],['From', 'To', 'P43243', 'MATR3'],['From', 'To', 'P43307', 'SSR1'],['From', 'To', 'P43378', 'PTPN9'],['From', 'To', 'P45983', 'MAPK8'],['From', 'To', 'P46013', 'MKI67'],['From', 'To'],['From', 'To', 'P46020', 'PHKA1'],['From', 'To', 'P46100', 'ATRX'],['From', 'To', 'P46937', 'YAP1'],['From', 'To', 'P46939', 'UTRN'],['From', 'To', 'P47870', 'GABRB2'],['From', 'To', 'P48058', 'GRIA4'],['From', 'To'],['From', 'To', 'P48681', 'NES'],['From', 'To'],['From', 'To', 'P48775', 'TDO2'],['From', 'To'],['From', 'To', 'P49321', 'NASP'],['From', 'To', 'P49454', 'CENPF'],['From', 'To', 'P49588', 'AARS'],['From', 'To', 'P49756', 'RBM25'],['From', 'To', 'P49790', 'NUP153'],['From', 'To', 'P49792', 'RANBP2'],['From', 'To', 'P49916', 'LIG3'],['From', 'To', 'P49959', 'MRE11'],['From', 'To', 'P50552', 'VASP'],['From', 'To', 'P50851', 'LRBA'],['From', 'To'],['From', 'To', 'P51116', 'FXR2'],['From', 'To', 'P51531', 'SMARCA2'],['From', 'To', 'P51610', 'HCFC1'],['From', 'To', 'P51784', 'USP11'],['From', 'To', 'P51790', 'CLCN3'],['From', 'To', 'P51800', 'CLCNKA'],['From', 'To', 'P51811', 'XK'],['From', 'To', 'P51957', 'NEK4'],['From', 'To', 'P52179', 'MYOM1'],['From', 'To'],['From', 'To', 'P52564', 'MAP2K6'],['From', 'To', 'P52594', 'AGFG1'],['From', 'To', 'P52701', 'MSH6'],['From', 'To', 'P53420', 'COL4A4'],['From', 'To', 'P53675', 'CLTCL1'],['From', 'To', 'P53804', 'TTC3'],['From', 'To', 'P53805', 'RCAN1'],['From', 'To'],['From', 'To', 'P53999', 'SUB1'],['From', 'To', 'P54132', 'BLM'],['From', 'To', 'P54198', 'HIRA'],['From', 'To', 'P54253', 'ATXN1'],['From', 'To', 'P54725', 'RAD23A'],['From', 'To', 'P55087', 'AQP4'],['From', 'To', 'P55145', 'MANF'],['From', 'To', 'P55196', 'AFDN'],['From', 'To'],['From', 'To', 'P55197', 'MLLT10'],['From', 'To', 'P55327', 'TPD52'],['From', 'To', 'P55957', 'BID'],['From', 'To', 'P56182', 'RRP1'],['From', 'To', 'P56545', 'CTBP2'],['From', 'To', 'P57060', 'RWDD2B'],['From', 'To', 'P57087', 'JAM2'],['From', 'To', 'P57764', 'GSDMD'],['From', 'To', 'P58107', 'EPPK1'],['From', 'To'],['From', 'To', 'P59542', 'TAS2R19'],['From', 'To', 'P60174', 'TPI1'],['From', 'To', 'P62277', 'RPS13'],['From', 'To', 'P62805', 'H4-16', 'P62805', 'H4C1', 'P62805', 'H4C11', 'P62805', 'H4C12', 'P62805', 'H4C13', 'P62805', 'H4C14', 'P62805', 'H4C15', 'P62805', 'H4C2', 'P62805', 'H4C3', 'P62805', 'H4C4', 'P62805', 'H4C5', 'P62805', 'H4C6', 'P62805', 'H4C8', 'P62805', 'H4C9'],['From', 'To', 'P62807', 'H2BC10', 'P62807', 'H2BC4', 'P62807', 'H2BC6', 'P62807', 'H2BC7', 'P62807', 'H2BC8'],['From', 'To', 'P63098', 'PPP3R1'],['From', 'To', 'P78345', 'RPP38'],['From', 'To', 'P78348', 'ASIC1'],['From', 'To', 'P78356', 'PIP4K2B'],['From', 'To', 'P81274', 'GPSM2'],['From', 'To', 'P81408', 'FAM189B'],['From', 'To', 'P82094', 'TMF1'],['From', 'To', 'P83111', 'LACTB'],['From', 'To', 'P98175', 'RBM10'],['From', 'To', 'Q00341', 'HDLBP'],['From', 'To', 'Q00536', 'CDK16'],['From', 'To', 'Q00587', 'CDC42EP1'],['From', 'To', 'Q00653', 'NFKB2'],['From', 'To', 'Q00722', 'PLCB2'],['From', 'To', 'Q00839', 'HNRNPU'],['From', 'To'],['From', 'To', 'Q00975', 'CACNA1B'],['From', 'To', 'Q01151', 'CD83'],['From', 'To', 'Q01196', 'RUNX1'],['From', 'To', 'Q01484', 'ANK2'],['From', 'To', 'Q01658', 'DR1'],['From', 'To', 'Q01804', 'OTUD4'],['From', 'To', 'Q01814', 'ATP2B2'],['From', 'To', 'Q01970', 'PLCB3'],['From', 'To', 'Q02224', 'CENPE'],['From', 'To'],['From', 'To', 'Q02505', 'MUC3A'],['From', 'To', 'Q02539', 'H1-1'],['From', 'To', 'Q02641', 'CACNB1'],['From', 'To', 'Q02750', 'MAP2K1'],['From', 'To', 'Q03001', 'DST'],['From', 'To', 'Q03060', 'CREM'],['From', 'To', 'Q03111', 'MLLT1'],['From', 'To', 'Q03135', 'CAV1'],['From', 'To', 'Q03164', 'KMT2A'],['From', 'To', 'Q03188', 'CENPC'],['From', 'To', 'Q03252', 'LMNB2'],['From', 'To', 'Q03468', 'ERCC6'],['From', 'To', 'Q03701', 'CEBPZ'],['From', 'To', 'Q04637', 'EIF4G1'],['From', 'To', 'Q04721', 'NOTCH2'],['From', 'To', 'Q05209', 'PTPN12'],['From', 'To', 'Q05397', 'PTK2'],['From', 'To', 'Q05586', 'GRIN1'],['From', 'To'],['From', 'To'],['From', 'To', 'Q06190', 'PPP2R3A'],['From', 'To', 'Q07157', 'TJP1'],['From', 'To'],['From', 'To', 'Q07866', 'KLC1'],['From', 'To'],['From', 'To', 'Q07912', 'TNK2'],['From', 'To', 'Q08174', 'PCDH1'],['From', 'To'],['From', 'To', 'Q08211', 'DHX9'],['From', 'To', 'Q08289', 'CACNB2'],['From', 'To', 'Q08378', 'GOLGA3'],['From', 'To'],['From', 'To', 'Q08945', 'SSRP1'],['From', 'To', 'Q09019', 'DMWD'],['From', 'To', 'Q09472', 'EP300'],['From', 'To', 'Q09666', 'AHNAK'],['From', 'To', 'Q0JRZ9', 'FCHO2'],['From', 'To', 'Q0VG06', 'FAAP100'],['From', 'To', 'Q0ZGT2', 'NEXN'],['From', 'To', 'Q10571', 'MN1'],['From', 'To'],['From', 'To', 'Q12769', 'NUP160'],['From', 'To', 'Q12778', 'FOXO1'],['From', 'To', 'Q12792', 'TWF1'],['From', 'To', 'Q12802', 'AKAP13'],['From', 'To', 'Q12830', 'BPTF'],['From', 'To', 'Q12851', 'MAP4K2'],['From', 'To', 'Q12857', 'NFIA'],['From', 'To', 'Q12879', 'GRIN2A'],['From', 'To', 'Q12906', 'ILF3'],['From', 'To'],['From', 'To'],['From', 'To', 'Q12959', 'DLG1'],['From', 'To'],['From', 'To', 'Q12983', 'BNIP3'],['From', 'To', 'Q13002', 'GRIK2'],['From', 'To', 'Q13017', 'ARHGAP5'],['From', 'To', 'Q13023', 'AKAP6'],['From', 'To', 'Q13029', 'PRDM2'],['From', 'To'],['From', 'To', 'Q13068', 'GAGE1'],['From', 'To', 'Q13085', 'ACACA'],['From', 'To', 'Q13098', 'GPS1'],['From', 'To', 'Q13111', 'CHAF1A'],['From', 'To', 'Q13112', 'CHAF1B'],['From', 'To', 'Q13123', 'IK'],['From', 'To', 'Q13129', 'RLF'],['From', 'To', 'Q13164', 'MAPK7'],['From', 'To', 'Q13239', 'SLA'],['From', 'To', 'Q13255', 'GRM1'],['From', 'To', 'Q13263', 'TRIM28'],['From', 'To', 'Q13330', 'MTA1'],['From', 'To', 'Q13351', 'KLF1'],['From', 'To', 'Q13409', 'DYNC1I2'],['From', 'To', 'Q13426', 'XRCC4'],['From', 'To', 'Q13428', 'TCOF1'],['From', 'To', 'Q13435', 'SF3B2'],['From', 'To', 'Q13439', 'GOLGA4'],['From', 'To', 'Q13459', 'MYO9B'],['From', 'To', 'Q13509', 'TUBB3'],['From', 'To', 'Q13546', 'RIPK1'],['From', 'To', 'Q13563', 'PKD2'],['From', 'To', 'Q13595', 'TRA2A'],['From', 'To', 'Q13596', 'SNX1'],['From', 'To', 'Q13620', 'CUL4B'],['From', 'To', 'Q13671', 'RIN1'],['From', 'To', 'Q13769', 'THOC5'],['From', 'To', 'Q13796', 'SHROOM2'],['From', 'To', 'Q13946', 'PDE7A'],['From', 'To', 'Q13972', 'RASGRF1'],['From', 'To', 'Q14004', 'CDK13'],['From', 'To'],['From', 'To', 'Q14149', 'MORC3'],['From', 'To', 'Q14156', 'EFR3A'],['From', 'To'],['From', 'To'],['From', 'To', 'Q14209', 'E2F2'],['From', 'To', 'Q14241', 'ELOA'],['From', 'To', 'Q14318', 'FKBP8'],['From', 'To', 'Q14444', 'CAPRIN1'],['From', 'To', 'Q14586', 'ZNF267'],['From', 'To', 'Q14669', 'TRIP12'],['From', 'To', 'Q14674', 'ESPL1'],['From', 'To', 'Q14676', 'MDC1'],['From', 'To', 'Q14677', 'CLINT1'],['From', 'To', 'Q14684', 'RRP1B'],['From', 'To', 'Q14693', 'LPIN1'],['From', 'To', 'Q14789', 'GOLGB1'],['From', 'To', 'Q14929', 'ZNF169'],['From', 'To', 'Q14980', 'NUMA1'],['From', 'To', 'Q15003', 'NCAPH'],['From', 'To', 'Q15021', 'NCAPD2'],['From', 'To', 'Q15032', 'R3HDM1'],['From', 'To'],['From', 'To', 'Q15047', 'SETDB1'],['From', 'To', 'Q15118', 'PDK1'],['From', 'To'],['From', 'To'],['From', 'To', 'Q15154', 'PCM1'],['From', 'To', 'Q15170', 'TCEAL1'],['From', 'To', 'Q15398', 'DLGAP5'],['From', 'To', 'Q15424', 'SAFB'],['From', 'To', 'Q15477', 'SKIV2L'],['From', 'To', 'Q15554', 'TERF2'],['From', 'To', 'Q15555', 'MAPRE2'],['From', 'To', 'Q15561', 'TEAD4'],['From', 'To', 'Q15569', 'TESK1'],['From', 'To'],['From', 'To', 'Q15643', 'TRIP11'],['From', 'To', 'Q15651', 'HMGN3'],['From', 'To', 'Q15746', 'MYLK'],['From', 'To', 'Q15825', 'CHRNA6'],['From', 'To', 'Q15906', 'VPS72'],['From', 'To', 'Q15911', 'ZFHX3'],['From', 'To', 'Q16186', 'ADRM1'],['From', 'To', 'Q16512', 'PKN1'],['From', 'To', 'Q16623', 'STX1A'],['From', 'To', 'Q16626', 'MEA1'],['From', 'To', 'Q16763', 'UBE2S'],['From', 'To', 'Q16816', 'PHKG1'],['From', 'To', 'Q16829', 'DUSP7'],['From', 'To', 'Q16881', 'TXNRD1'],['From', 'To', 'Q16890', 'TPD52L1'],['From', 'To', 'Q17RV1', 'CEP152'],['From', 'To', 'Q1KMD3', 'HNRNPUL2'],['From', 'To', 'Q1MSJ5', 'CSPP1'],['From', 'To', 'Q2KHR3', 'QSER1'],['From', 'To', 'Q2VCE5', 'UCA1'],['From', 'To', 'Q32MZ4', 'LRRFIP1'],['From', 'To', 'Q32NC0', 'C18orf21'],['From', 'To', 'Q3KQU3', 'MAP7D1'],['From', 'To', 'Q3KR37', 'GRAMD1B'],['From', 'To', 'Q3MII6', 'TBC1D25'],['From', 'To', 'Q3YEC7', 'RABL6'],['From', 'To', 'Q4G0Z9', 'MCMDC2'],['From', 'To', 'Q4W5G0', 'TIGD2'],['From', 'To', 'Q52WX2', 'SBK1'],['From', 'To', 'Q53SF7', 'COBLL1'],['From', 'To', 'Q58A45', 'PAN3'],['From', 'To', 'Q58A63', 'PRUNE2'],['From', 'To'],['From', 'To', 'Q5BJE1', 'CCDC178'],['From', 'To', 'Q5BKX5', 'C19orf54'],['From', 'To', 'Q5BKY9', 'FAM133B'],['From', 'To', 'Q5BKZ1', 'ZNF326'],['From', 'To', 'Q5CZA4', 'SLC35E2A', 'Q5CZA4', 'SLC35E2B'],['From', 'To', 'Q5CZH9', 'ANK3'],['From', 'To', 'Q5H9R7', 'PPP6R3'],['From', 'To', 'Q5JRA6', 'MIA3'],['From', 'To', 'Q5JSH3', 'WDR44'],['From', 'To', 'Q5JSZ5', 'PRRC2B'],['From', 'To', 'Q5M775', 'SPECC1'],['From', 'To', 'Q5SVZ6', 'ZMYM1'],['From', 'To', 'Q5T0D9', 'TPRG1L'],['From', 'To', 'Q5T1V6', 'DDX59'],['From', 'To', 'Q5T200', 'ZC3H13'],['From', 'To', 'Q5T4C3', 'ODF2'],['From', 'To', 'Q5T5P2', 'KIAA1217'],['From', 'To', 'Q5T5U3', 'ARHGAP21'],['From', 'To', 'Q5T7V8', 'GORAB'],['From', 'To', 'Q5T8D3', 'ACBD5'],['From', 'To', 'Q5T8I3', 'FAM102B'],['From', 'To'],['From', 'To', 'Q5T9S5', 'CCDC18'],['From', 'To', 'Q5TDH0', 'DDI2'],['From', 'To', 'Q5TG08', 'FAM227A'],['From', 'To', 'Q5VT52', 'RPRD2'],['From', 'To', 'Q5VUA4', 'ZNF318'],['From', 'To', 'Q5VVJ2', 'MYSM1'],['From', 'To', 'Q5VWQ8', 'DAB2IP'],['From', 'To', 'Q5VYS8', 'TUT7'],['From', 'To', 'Q5VZ89', 'DENND4C'],['From', 'To', 'Q5VZL5', 'ZMYM4'],['From', 'To', 'Q658Y4', 'FAM91A1'],['From', 'To', 'Q66K43', 'COL27A1'],['From', 'To', 'Q676U5', 'ATG16L1'],['From', 'To', 'Q68D20', 'PMS2CL'],['From', 'To', 'Q68DE3', 'USF3'],['From', 'To', 'Q68DN1', 'C2orf16'],['From', 'To', 'Q69YH5', 'CDCA2'],['From', 'To', 'Q69YQ0', 'SPECC1L'],['From', 'To', 'Q6BCY4', 'CYB5R2'],['From', 'To', 'Q6DN12', 'MCTP2'],['From', 'To', 'Q6DSU6', 'GTF2IRD1'],['From', 'To', 'Q6EMB2', 'TTLL5'],['From', 'To', 'Q6EMK4', 'VASN'],['From', 'To', 'Q6FIF0', 'ZFAND6'],['From', 'To', 'Q6KC79', 'NIPBL'],['From', 'To'],['From', 'To', 'Q6NUK4', 'REEP3'],['From', 'To', 'Q6P0N0', 'MIS18BP1'],['From', 'To', 'Q6P0Q8', 'MAST2'],['From', 'To', 'Q6P1N0', 'CC2D1A'],['From', 'To', 'Q6P2H3', 'CEP85'],['From', 'To', 'Q6P4R8', 'NFRKB'],['From', 'To', 'Q6P5Z2', 'PKN3'],['From', 'To'],['From', 'To', 'Q6P995', 'FAM171B'],['From', 'To', 'Q6PCT2', 'FBXL19'],['From', 'To', 'Q6PGN9', 'PSRC1'],['From', 'To', 'Q6PGQ7', 'BORA'],['From', 'To', 'Q6PJ61', 'FBXO46'],['From', 'To', 'Q6PKG0', 'LARP1'],['From', 'To'],['From', 'To'],['From', 'To', 'Q6T4R5', 'NHS'],['From', 'To', 'Q6UB99', 'ANKRD11'],['From', 'To', 'Q6UUV9', 'CRTC1'],['From', 'To', 'Q6UVJ0', 'SASS6'],['From', 'To', 'Q6V1X1', 'DPP8'],['From', 'To', 'Q6VMQ6', 'ATF7IP'],['From', 'To', 'Q6W2J9', 'BCOR'],['From', 'To', 'Q6WCQ1', 'MPRIP'],['From', 'To', 'Q6WKZ4', 'RAB11FIP1'],['From', 'To', 'Q6XZF7', 'DNMBP'],['From', 'To'],['From', 'To'],['From', 'To', 'Q6ZMI0', 'PPP1R21'],['From', 'To', 'Q6ZNB6', 'NFXL1'],['From', 'To', 'Q6ZNC4', 'ZNF704'],['From', 'To'],['From', 'To'],['From', 'To'],['From', 'To', 'Q6ZU80', 'CEP128'],['From', 'To', 'Q6ZVG3', 'TTYH3'],['From', 'To', 'Q70CQ2', 'USP34'],['From', 'To', 'Q70E73', 'RAPH1'],['From', 'To', 'Q70EL4', 'USP43'],['From', 'To', 'Q70JA7', 'CHSY3'],['From', 'To', 'Q71RC2', 'LARP4'],['From', 'To'],['From', 'To', 'Q7KZ85', 'SUPT6H'],['From', 'To'],['From', 'To', 'Q7L2J0', 'MEPCE'],['From', 'To', 'Q7LBC6', 'KDM3B'],['From', 'To', 'Q7RTP6', 'MICAL3'],['From', 'To', 'Q7Z3B3', 'KANSL1'],['From', 'To', 'Q7Z3D4', 'LYSMD3'],['From', 'To', 'Q7Z3T8', 'ZFYVE16'],['From', 'To', 'Q7Z3V4', 'UBE3B'],['From', 'To', 'Q7Z434', 'MAVS'],['From', 'To', 'Q7Z460', 'CLASP1'],['From', 'To'],['From', 'To', 'Q7Z5L4', 'SPATA19'],['From', 'To', 'Q7Z620', 'WWC2'],['From', 'To', 'Q7Z6B7', 'SRGAP1'],['From', 'To', 'Q7Z6E9', 'RBBP6'],['From', 'To', 'Q7Z6J0', 'SH3RF1'],['From', 'To', 'Q7Z6Z7', 'HUWE1'],['From', 'To', 'Q7Z7K6', 'CENPV'],['From', 'To', 'Q86SJ2', 'AMIGO2'],['From', 'To', 'Q86SQ0', 'PHLDB2'],['From', 'To', 'Q86TC9', 'MYPN'],['From', 'To', 'Q86TU1', 'ZNF839'],['From', 'To', 'Q86TV6', 'TTC7B'],['From', 'To', 'Q86U44', 'METTL3'],['From', 'To', 'Q86UP2', 'KTN1'],['From', 'To', 'Q86UR5', 'RIMS1'],['From', 'To', 'Q86UW7', 'CADPS2'],['From', 'To', 'Q86V48', 'LUZP1'],['From', 'To', 'Q86VI4', 'LAPTM4B'],['From', 'To', 'Q86VQ1', 'GLCCI1'],['From', 'To'],['From', 'To', 'Q86XJ1', 'GAS2L3'],['From', 'To', 'Q86XL3', 'ANKLE2'],['From', 'To', 'Q86Y07', 'VRK2'],['From', 'To', 'Q86YR5', 'GPSM1'],['From', 'To', 'Q86YS7', 'C2CD5'],['From', 'To'],['From', 'To', 'Q86Z20', 'CCDC125'],['From', 'To', 'Q8IUD2', 'ERC1'],['From', 'To', 'Q8IUI4', 'SNX29P2'],['From', 'To', 'Q8IVL1', 'NAV2'],['From', 'To', 'Q8IVT2', 'MISP'],['From', 'To', 'Q8IVT5', 'KSR1'],['From', 'To', 'Q8IW35', 'CEP97'],['From', 'To', 'Q8IWJ2', 'GCC2'],['From', 'To', 'Q8IWK6', 'ADGRA3'],['From', 'To', 'Q8IWR0', 'ZC3H7A'],['From', 'To'],['From', 'To', 'Q8IWW6', 'ARHGAP12'],['From', 'To', 'Q8IWX7', 'UNC45B'],['From', 'To', 'Q8IX01', 'SUGP2'],['From', 'To', 'Q8IX07', 'ZFPM1'],['From', 'To', 'Q8IX15', 'HOMEZ'],['From', 'To', 'Q8IXJ6', 'SIRT2'],['From', 'To', 'Q8IXZ2', 'ZC3H3'],['From', 'To'],['From', 'To', 'Q8IY81', 'FTSJ3'],['From', 'To', 'Q8IYB3', 'SRRM1'],['From', 'To', 'Q8IYH5', 'ZZZ3'],['From', 'To', 'Q8IZ21', 'PHACTR4'],['From', 'To', 'Q8IZD4', 'DCP1B'],['From', 'To', 'Q8IZH2', 'XRN1'],['From', 'To', 'Q8IZL8', 'PELP1'],['From', 'To', 'Q8IZP2', 'ST13P4'],['From', 'To', 'Q8IZQ1', 'WDFY3'],['From', 'To', 'Q8N108', 'MIER1'],['From', 'To', 'Q8N163', 'CCAR2'],['From', 'To'],['From', 'To', 'Q8N2M8', 'CLASRP'],['From', 'To', 'Q8N350', 'CBARP'],['From', 'To', 'Q8N3D4', 'EHBP1L1'],['From', 'To', 'Q8N3L3', 'TXLNB'],['From', 'To', 'Q8N3V7', 'SYNPO'],['From', 'To'],['From', 'To', 'Q8N3X1', 'FNBP4'],['From', 'To', 'Q8N3Z6', 'ZCCHC7'],['From', 'To', 'Q8N414', 'PGBD5'],['From', 'To', 'Q8N448', 'LNX2'],['From', 'To', 'Q8N488', 'RYBP'],['From', 'To', 'Q8N490', 'PNKD'],['From', 'To', 'Q8N554', 'ZNF276'],['From', 'To', 'Q8N573', 'OXR1'],['From', 'To', 'Q8N5A5', 'ZGPAT'],['From', 'To', 'Q8N5C8', 'TAB3'],['From', 'To', 'Q8N655', 'LCOR'],['From', 'To', 'Q8N6H7', 'ARFGAP2'],['From', 'To', 'Q8N6L1', 'KRTCAP2'],['From', 'To', 'Q8N6T3', 'ARFGAP1'],['From', 'To'],['From', 'To', 'Q8N8S7', 'ENAH'],['From', 'To'],['From', 'To', 'Q8N9M1', 'C19orf47'],['From', 'To', 'Q8N9T8', 'KRI1'],['From', 'To'],['From', 'To'],['From', 'To', 'Q8NBF6', 'AVL9'],['From', 'To', 'Q8NBU5', 'ATAD1'],['From', 'To'],['From', 'To', 'Q8NCN4', 'RNF169'],['From', 'To', 'Q8ND56', 'LSM14A'],['From', 'To', 'Q8NDD1', 'C1orf131'],['From', 'To', 'Q8NDT2', 'RBM15B'],['From', 'To', 'Q8NE35', 'CPEB3'],['From', 'To'],['From', 'To', 'Q8NEL9', 'DDHD1'],['From', 'To', 'Q8NEM2', 'SHCBP1'],['From', 'To', 'Q8NET4', 'RTL9'],['From', 'To', 'Q8NEY1', 'NAV1'],['From', 'To'],['From', 'To', 'Q8NEZ4', 'KMT2C'],['From', 'To', 'Q8NFA0', 'USP32'],['From', 'To', 'Q8NFI4', 'ST13P5'],['From', 'To', 'Q8NFQ8', 'TOR1AIP2'],['From', 'To', 'Q8NFT2', 'STEAP2'],['From', 'To', 'Q8NG31', 'KNL1'],['From', 'To', 'Q8NHG8', 'ZNRF2'],['From', 'To', 'Q8NHM5', 'KDM2B'],['From', 'To', 'Q8NHP6', 'MOSPD2'],['From', 'To', 'Q8NI08', 'NCOA7'],['From', 'To', 'Q8NI27', 'THOC2'],['From', 'To', 'Q8TAA9', 'VANGL1'],['From', 'To', 'Q8TBF4', 'ZCRB1'],['From', 'To', 'Q8TBZ0', 'CCDC110'],['From', 'To', 'Q8TCN5', 'ZNF507'],['From', 'To', 'Q8TCU4', 'ALMS1'],['From', 'To', 'Q8TDJ6', 'DMXL2'],['From', 'To', 'Q8TEQ0', 'SNX29'],['From', 'To', 'Q8TEQ6', 'GEMIN5'],['From', 'To', 'Q8TEV9', 'SMCR8'],['From', 'To', 'Q8TEW0', 'PARD3'],['From', 'To', 'Q8TF72', 'SHROOM3'],['From', 'To', 'Q8WU79', 'SMAP2'],['From', 'To', 'Q8WUA7', 'TBC1D22A'],['From', 'To', 'Q8WUB8', 'PHF10'],['From', 'To', 'Q8WUF5', 'PPP1R13L'],['From', 'To', 'Q8WUQ7', 'CACTIN'],['From', 'To', 'Q8WUR7', 'C15orf40'],['From', 'To', 'Q8WUX9', 'CHMP7'],['From', 'To', 'Q8WVM7', 'STAG1'],['From', 'To', 'Q8WVT3', 'TRAPPC12'],['From', 'To', 'Q8WWI1', 'LMO7'],['From', 'To'],['From', 'To'],['From', 'To', 'Q8WWQ0', 'PHIP'],['From', 'To', 'Q8WXA3', 'RUFY2'],['From', 'To', 'Q8WXE0', 'CASKIN2'],['From', 'To', 'Q8WXK8', 'DST'],['From', 'To', 'Q8WYP5', 'AHCTF1'],['From', 'To'],['From', 'To', 'Q8WZ64', 'ARAP2'],['From', 'To', 'Q92508', 'PIEZO1'],['From', 'To', 'Q92541', 'RTF1'],['From', 'To', 'Q92545', 'TMEM131'],['From', 'To', 'Q92576', 'PHF3'],['From', 'To', 'Q92610', 'ZNF592'],['From', 'To', 'Q92613', 'JADE3'],['From', 'To'],['From', 'To', 'Q92615', 'LARP4B'],['From', 'To', 'Q92629', 'SGCD'],['From', 'To', 'Q92738', 'USP6NL'],['From', 'To', 'Q92752', 'TNR'],['From', 'To', 'Q92764', 'KRT35'],['From', 'To', 'Q92766', 'RREB1'],['From', 'To', 'Q92777', 'SYN2'],['From', 'To', 'Q92793', 'CREBBP'],['From', 'To', 'Q92841', 'DDX17'],['From', 'To', 'Q92887', 'ABCC2'],['From', 'To', 'Q92888', 'ARHGEF1'],['From', 'To', 'Q92890', 'UFD1'],['From', 'To', 'Q92908', 'GATA6'],['From', 'To', 'Q92922', 'SMARCC1'],['From', 'To', 'Q92925', 'SMARCD2'],['From', 'To', 'Q93008', 'USP9X'],['From', 'To', 'Q93052', 'LPP'],['From', 'To', 'Q93074', 'MED12'],['From', 'To', 'Q93075', 'TATDN2'],['From', 'To', 'Q969Q5', 'RAB24'],['From', 'To', 'Q96AP0', 'ACD'],['From', 'To', 'Q96B23', 'C18orf25'],['From', 'To', 'Q96BV0', 'ZNF775'],['From', 'To', 'Q96BZ4', 'PLD4'],['From', 'To', 'Q96C24', 'SYTL4'],['From', 'To', 'Q96C45', 'ULK4'],['From', 'To', 'Q96D71', 'REPS1'],['From', 'To', 'Q96DM3', 'RMC1'],['From', 'To', 'Q96DT7', 'ZBTB10'],['From', 'To', 'Q96E09', 'FAM122A'],['From', 'To', 'Q96ER3', 'SAAL1'],['From', 'To', 'Q96F63', 'CCDC97'],['From', 'To', 'Q96GS4', 'BORCS6'],['From', 'To', 'Q96GX8', 'C16orf74'],['From', 'To', 'Q96GY3', 'LIN37'],['From', 'To', 'Q96I24', 'FUBP3'],['From', 'To', 'Q96IY1', 'NSL1'],['From', 'To', 'Q96JB1', 'DNAH8'],['From', 'To', 'Q96JM2', 'ZNF462'],['From', 'To', 'Q96JM3', 'CHAMP1'],['From', 'To', 'Q96JN0', 'LCOR'],['From', 'To', 'Q96K37', 'SLC35E1'],['From', 'To', 'Q96K76', 'USP47'],['From', 'To', 'Q96KQ7', 'EHMT2'],['From', 'To'],['From', 'To', 'Q96KW2', 'POM121L2'],['From', 'To', 'Q96L73', 'NSD1'],['From', 'To', 'Q96L91', 'EP400'],['From', 'To', 'Q96LW7', 'CARD19'],['From', 'To', 'Q96MF7', 'NSMCE2'],['From', 'To', 'Q96MT7', 'CFAP44'],['From', 'To', 'Q96MW7', 'TIGD1'],['From', 'To'],['From', 'To', 'Q96NB3', 'ZNF830'],['From', 'To', 'Q96P48', 'ARAP1'],['From', 'To'],['From', 'To', 'Q96PE2', 'ARHGEF17'],['From', 'To', 'Q96PK2', 'MACF1'],['From', 'To', 'Q96PY5', 'FMNL2'],['From', 'To', 'Q96Q15', 'SMG1'],['From', 'To', 'Q96QE2', 'SLC2A13'],['From', 'To', 'Q96RG2', 'PASK'],['From', 'To', 'Q96RL7', 'VPS13A'],['From', 'To', 'Q96RT1', 'ERBIN'],['From', 'To', 'Q96RU3', 'FNBP1'],['From', 'To', 'Q96RV3', 'PCNX1'],['From', 'To', 'Q96RY5', 'CRAMP1'],['From', 'To', 'Q96SB3', 'PPP1R9B'],['From', 'To', 'Q96SB4', 'SRPK1'],['From', 'To', 'Q96SB8', 'SMC6'],['From', 'To', 'Q96ST3', 'SIN3A'],['From', 'To', 'Q96T23', 'RSF1'],['From', 'To', 'Q96T25', 'ZIC5'],['From', 'To', 'Q96TA1', 'NIBAN2'],['From', 'To', 'Q99549', 'MPHOSPH8'],['From', 'To', 'Q99567', 'NUP88'],['From', 'To', 'Q99569', 'PKP4'],['From', 'To', 'Q99572', 'P2RX7'],['From', 'To', 'Q99590', 'SCAF11'],['From', 'To', 'Q99615', 'DNAJC7'],['From', 'To', 'Q99626', 'CDX2'],['From', 'To', 'Q99638', 'RAD9A'],['From', 'To', 'Q99650', 'OSMR'],['From', 'To', 'Q99661', 'KIF2C'],['From', 'To', 'Q99684', 'GFI1'],['From', 'To', 'Q99700', 'ATXN2'],['From', 'To', 'Q99704', 'DOK1'],['From', 'To', 'Q99717', 'SMAD5'],['From', 'To', 'Q99747', 'NAPG'],['From', 'To', 'Q99871', 'HAUS7'],['From', 'To', 'Q99873', 'PRMT1'],['From', 'To', 'Q99952', 'PTPN18'],['From', 'To', 'Q99956', 'DUSP9'],['From', 'To', 'Q99962', 'SH3GL2'],['From', 'To', 'Q99966', 'CITED1'],['From', 'To', 'Q99986', 'VRK1'],['From', 'To', 'Q99996', 'AKAP9'],['From', 'To', 'Q9BQ67', 'GRWD1'],['From', 'To', 'Q9BQ70', 'TCF25'],['From', 'To', 'Q9BQA9', 'CYBC1'],['From', 'To', 'Q9BQL6', 'FERMT1'],['From', 'To', 'Q9BRD0', 'BUD13'],['From', 'To', 'Q9BRG2', 'SH2D3A'],['From', 'To', 'Q9BRH9', 'ZNF251'],['From', 'To', 'Q9BRN9', 'TM2D3'],['From', 'To', 'Q9BSE2', 'TMEM79'],['From', 'To', 'Q9BTA9', 'WAC'],['From', 'To', 'Q9BTC0', 'DIDO1'],['From', 'To', 'Q9BTX1', 'NDC1'],['From', 'To', 'Q9BU19', 'ZNF692'],['From', 'To', 'Q9BVC5', 'C2orf49'],['From', 'To', 'Q9BVI0', 'PHF20'],['From', 'To', 'Q9BVS4', 'RIOK2'],['From', 'To', 'Q9BW34', 'EEF1D'],['From', 'To', 'Q9BW85', 'YJU2'],['From', 'To', 'Q9BX66', 'SORBS1'],['From', 'To'],['From', 'To', 'Q9BXF6', 'RAB11FIP5'],['From', 'To', 'Q9BXK5', 'BCL2L13'],['From', 'To', 'Q9BXS6', 'NUSAP1'],['From', 'To', 'Q9BY15', 'ADGRE3'],['From', 'To', 'Q9BY89', 'KIAA1671'],['From', 'To', 'Q9BYJ9', 'YTHDF1'],['From', 'To', 'Q9BYX2', 'TBC1D2'],['From', 'To', 'Q9BZB8', 'CPEB1'],['From', 'To', 'Q9BZC7', 'ABCA2'],['From', 'To', 'Q9BZQ8', 'NIBAN1'],['From', 'To', 'Q9BZV1', 'UBXN6'],['From', 'To'],['From', 'To', 'Q9C0C2', 'TNKS1BP1'],['From', 'To', 'Q9C0E8', 'LNPK'],['From', 'To', 'Q9C0G0', 'ZNF407'],['From', 'To', 'Q9GZS1', 'POLR1E'],['From', 'To', 'Q9GZV8', 'PRDM14'],['From', 'To', 'Q9GZZ9', 'UBA5'],['From', 'To'],['From', 'To', 'Q9H0H3', 'KLHL25'],['From', 'To', 'Q9H1B7', 'IRF2BPL'],['From', 'To', 'Q9H1K1', 'ISCU'],['From', 'To', 'Q9H2D6', 'TRIOBP'],['From', 'To', 'Q9H2P0', 'ADNP'],['From', 'To', 'Q9H2U1', 'DHX36'],['From', 'To', 'Q9H2Y7', 'ZNF106'],['From', 'To', 'Q9H3P2', 'NELFA'],['From', 'To', 'Q9H4X1', 'RGCC'],['From', 'To', 'Q9H5I1', 'SUV39H2'],['From', 'To', 'Q9H5U6', 'ZCCHC4'],['From', 'To', 'Q9H6S0', 'YTHDC2'],['From', 'To', 'Q9H6Y2', 'WDR55'],['From', 'To', 'Q9H728', 'hCG_1744006'],['From', 'To', 'Q9H773', 'DCTPP1'],['From', 'To', 'Q9H7D0', 'DOCK5'],['From', 'To', 'Q9H7E2', 'TDRD3'],['From', 'To', 'Q9H7N4', 'SCAF1'],['From', 'To', 'Q9H814', 'PHAX'],['From', 'To'],['From', 'To', 'Q9H910', 'JPT2'],['From', 'To', 'Q9H956', 'RFX7'],['From', 'To', 'Q9H9B1', 'EHMT1'],['From', 'To', 'Q9H9J4', 'USP42'],['From', 'To', 'Q9HAH7', 'FBRS'],['From', 'To', 'Q9HAS0', 'C17orf75'],['From', 'To', 'Q9HAW4', 'CLSPN'],['From', 'To', 'Q9HBG4', 'ATP6V0A4'],['From', 'To', 'Q9HBL0', 'TNS1'],['From', 'To', 'Q9HBL6', 'LRTM1'],['From', 'To', 'Q9HBZ2', 'ARNT2'],['From', 'To', 'Q9HC35', 'EML4'],['From', 'To', 'Q9HCD5', 'NCOA5'],['From', 'To', 'Q9HCE1', 'MOV10'],['From', 'To', 'Q9HCF4', 'RNF213'],['From', 'To', 'Q9HCK8', 'CHD8'],['From', 'To', 'Q9HCU4', 'CELSR2'],['From', 'To', 'Q9HD67', 'MYO10'],['From', 'To', 'Q9NPA5', 'ZFP64'],['From', 'To', 'Q9NPQ8', 'RIC8A'],['From', 'To', 'Q9NPR2', 'SEMA4B'],['From', 'To', 'Q9NQ55', 'PPAN'],['From', 'To', 'Q9NQA5', 'TRPV5'],['From', 'To', 'Q9NQC7', 'CYLD'],['From', 'To', 'Q9NQV6', 'PRDM10'],['From', 'To', 'Q9NQW6', 'ANLN'],['From', 'To', 'Q9NR09', 'BIRC6'],['From', 'To', 'Q9NRA0', 'SPHK2'],['From', 'To', 'Q9NRA8', 'EIF4ENIF1'],['From', 'To', 'Q9NRF8', 'CTPS2'],['From', 'To', 'Q9NRF9', 'POLE3'],['From', 'To', 'Q9NS56', 'TOPORS'],['From', 'To', 'Q9NS85', 'CA10'],['From', 'To', 'Q9NSC7', 'ST6GALNAC1'],['From', 'To'],['From', 'To', 'Q9NSU2', 'TREX1'],['From', 'To', 'Q9NSV0', 'RBM33'],['From', 'To', 'Q9NUD5', 'ZCCHC3'],['From', 'To', 'Q9NUQ3', 'TXLNG'],['From', 'To', 'Q9NUW8', 'TDP1'],['From', 'To', 'Q9NVP4', 'DZANK1'],['From', 'To', 'Q9NVT9', 'ARMC1'],['From', 'To', 'Q9NVV0', 'TMEM38B'],['From', 'To', 'Q9NXL9', 'MCM9'],['From', 'To'],['From', 'To', 'Q9NXZ1', 'SAGE1'],['From', 'To', 'Q9NY61', 'AATF'],['From', 'To', 'Q9NYF8', 'BCLAF1'],['From', 'To', 'Q9NYU2', 'UGGT1'],['From', 'To', 'Q9NYV4', 'CDK12'],['From', 'To', 'Q9NYZ3', 'GTSE1'],['From', 'To', 'Q9NZ38', 'IDI2-AS1'],['From', 'To', 'Q9NZ56', 'FMN2'],['From', 'To', 'Q9NZC7', 'WWOX'],['From', 'To', 'Q9NZJ5', 'EIF2AK3'],['From', 'To', 'Q9NZL4', 'HSPBP1'],['From', 'To', 'Q9NZM1', 'MYOF'],['From', 'To', 'Q9NZM3', 'ITSN2'],['From', 'To', 'Q9NZM5', 'NOP53'],['From', 'To', 'Q9NZN8', 'CNOT2'],['From', 'To', 'Q9NZV8', 'KCND2'],['From', 'To', 'Q9P0P8', 'MTRES1'],['From', 'To', 'Q9P1T7', 'MDFIC'],['From', 'To', 'Q9P1Y5', 'CAMSAP3'],['From', 'To', 'Q9P207', 'GAPVD1'],['From', 'To', 'Q9P227', 'ARHGAP23'],['From', 'To', 'Q9P242', 'NYAP2'],['From', 'To', 'Q9P275', 'USP36'],['From', 'To', 'Q9P2D1', 'CHD7'],['From', 'To', 'Q9P2F8', 'SIPA1L2'],['From', 'To', 'Q9P2G1', 'ANKIB1'],['From', 'To', 'Q9P2G4', 'MAP10'],['From', 'To', 'Q9P2H5', 'USP35'],['From', 'To', 'Q9P2N5', 'RBM27'],['From', 'To', 'Q9P2Z0', 'THAP10'],['From', 'To', 'Q9UBD8', 'CLN3'],['From', 'To'],['From', 'To', 'Q9UBL6', 'CPNE7'],['From', 'To', 'Q9UDY4', 'DNAJB4'],['From', 'To', 'Q9UER7', 'DAXX'],['From', 'To', 'Q9UGR2', 'ZC3H7B'],['From', 'To', 'Q9UGU0', 'TCF20'],['From', 'To', 'Q9UGY1', 'NOL12'],['From', 'To', 'Q9UHB7', 'AFF4'],['From', 'To', 'Q9UHW9', 'SLC12A6'],['From', 'To', 'Q9UI08', 'EVL'],['From', 'To', 'Q9UIF8', 'BAZ2B'],['From', 'To', 'Q9UIF9', 'BAZ2A'],['From', 'To', 'Q9UIG0', 'BAZ1B'],['From', 'To', 'Q9UIU6', 'SIX4'],['From', 'To', 'Q9UJ41', 'RABGEF1'],['From', 'To', 'Q9UJQ4', 'SALL4'],['From', 'To', 'Q9UJX4', 'ANAPC5'],['From', 'To', 'Q9UKF7', 'PITPNC1'],['From', 'To'],['From', 'To', 'Q9UKJ3', 'GPATCH8'],['From', 'To', 'Q9UKL0', 'RCOR1'],['From', 'To', 'Q9UKS7', 'IKZF2'],['From', 'To'],['From', 'To', 'Q9UKV3', 'ACIN1'],['From', 'To', 'Q9UKV5', 'AMFR'],['From', 'To', 'Q9UKY1', 'ZHX1'],['From', 'To', 'Q9UL51', 'HCN2'],['From', 'To', 'Q9UL60', 'HSSG1'],['From', 'To', 'Q9ULE0', 'WWC3'],['From', 'To', 'Q9ULH7', 'MRTFB'],['From', 'To', 'Q9ULI4', 'KIF26A'],['From', 'To', 'Q9ULJ3', 'ZBTB21'],['From', 'To', 'Q9ULJ7', 'ANKRD50'],['From', 'To', 'Q9ULJ8', 'PPP1R9A'],['From', 'To', 'Q9ULL5', 'PRR12'],['From', 'To', 'Q9ULT8', 'HECTD1'],['From', 'To', 'Q9ULV0', 'MYO5B'],['From', 'To', 'Q9ULW0', 'TPX2'],['From', 'To', 'Q9UM00', 'TMCO1'],['From', 'To', 'Q9UMQ3', 'BARX2'],['From', 'To', 'Q9UMR2', 'DDX19B'],['From', 'To', 'Q9UN30', 'SCML1'],['From', 'To', 'Q9UN79', 'SOX13'],['From', 'To', 'Q9UN86', 'G3BP2'],['From', 'To'],['From', 'To', 'Q9UNE0', 'EDAR'],['From', 'To', 'Q9UNN4', 'GTF2A1L'],['From', 'To', 'Q9UNX4', 'WDR3'],['From', 'To', 'Q9UNZ2', 'NSFL1C'],['From', 'To', 'Q9UPN4', 'CEP131'],['From', 'To', 'Q9UPN7', 'PPP6R1'],['From', 'To', 'Q9UPQ0', 'LIMCH1'],['From', 'To', 'Q9UPQ6', 'PRRC2C'],['From', 'To', 'Q9UPS8', 'ANKRD26'],['From', 'To', 'Q9UPU5', 'USP24'],['From', 'To', 'Q9UPX8', 'SHANK2'],['From', 'To', 'Q9UPY3', 'DICER1'],['From', 'To', 'Q9UQ35', 'SRRM2'],['From', 'To', 'Q9UQ88', 'CDK11A'],['From', 'To', 'Q9UQR0', 'SCML2'],['From', 'To'],['From', 'To', 'Q9Y2H5', 'PLEKHA6'],['From', 'To', 'Q9Y2H6', 'FNDC3A'],['From', 'To', 'Q9Y2K1', 'ZBTB1'],['From', 'To', 'Q9Y2K3', 'MYH15'],['From', 'To', 'Q9Y2L6', 'FRMD4B'],['From', 'To', 'Q9Y2N3', 'POM121', 'Q9Y2N3', 'POM121C'],['From', 'To', 'Q9Y2R4', 'DDX52'],['From', 'To', 'Q9Y2W1', 'THRAP3'],['From', 'To', 'Q9Y314', 'NOSIP'],['From', 'To', 'Q9Y3F4', 'STRAP'],['From', 'To', 'Q9Y3Q8', 'TSC22D4'],['From', 'To', 'Q9Y3T9', 'NOC2L'],['From', 'To', 'Q9Y450', 'HBS1L'],['From', 'To', 'Q9Y462', 'ZNF711'],['From', 'To', 'Q9Y496', 'KIF3A'],['From', 'To', 'Q9Y4A5', 'TRRAP'],['From', 'To', 'Q9Y4B5', 'MTCL1'],['From', 'To', 'Q9Y4F3', 'MARF1'],['From', 'To', 'Q9Y4K1', 'CRYBG1'],['From', 'To', 'Q9Y580', 'RBM7'],['From', 'To', 'Q9Y5P4', 'CERT'],['From', 'To', 'Q9Y608', 'LRRFIP2'],['From', 'To', 'Q9Y653', 'ADGRG1'],['From', 'To', 'Q9Y679', 'AUP1'],['From', 'To', 'Q9Y6D5', 'ARFGEF2'],['From', 'To', 'Q9Y6D6', 'ARFGEF1'],['From', 'To', 'Q9Y6D9', 'MAD1L1'],['From', 'To', 'Q9Y6G9', 'DYNC1LI1'],['From', 'To', 'Q9Y6K1', 'DNMT3A'],['From', 'To', 'Q9Y6M4', 'CSNK1G3'],['From', 'To'],['From', 'To', 'Q9Y6M7', 'SLC4A7'],['From', 'To', 'Q9Y6N7', 'ROBO1']]

In [ ]:
# Only attempt to translate unambiguous accession IDs
# Each sub-list should have length 4, e.g. "['From', 'To', 'Q9Y6W5', 'WASF2']"

genedict = {}

for i in results2: # i = one substrate and all of its possible translations
    if( len( i ) ) == 4 : # Lists of this length have only one, unambiguous translation option
        genedict[ str( i[2] ) ] = str( i[3] ) # Add to dictionary

In [ ]:
# Create gene ID column for database

gene_ids = []

for n, i in enumerate( phosphosite_2_df.acc ): # Check substrate acc IDs
    if str( i ) in genedict.keys(): # If they are in the dictionary
        gene_ids.append( genedict.get( str(i) ) ) # Add the corresponding
        # gene ID to the column
    else: # Otherwise append nothing
        gene_ids.append( "" )

gene_ids = pd.Series( gene_ids )
phosphosite_2_df = phosphosite_2_df.assign( SUB_GENE = gene_ids )

Translate the kinase IDs to UniProt IDs, where possible

In [ ]:
# Make a list of kinases from phosphosite_2_df
# Remove duplicates
# Convert them to uppercase

kinases_ = phosphosite_2_df.kinases

kinases_ = list( kinases_.drop_duplicates() )

kinases = []

for n, i in enumerate( kinases_ ):
    if len( i ) > 1:
        kinases.append( str( i ) )

In [ ]:
# Make an empty dictionary for storing kinase aliases
# The keys will be kinase names from phosphosite_2_df
# The values will be the UniProt accession IDs from
# kinases_df

kinase_dict = {}

In [ ]:
# Check five columns in kinases_df for each kinase ID in phosphosite_2_df
# If kinase is found, store the corresponding UniProt ID in the dictionary
# and remove it from the list "kinases"

# For some reason the following five loops need to be executed three times
# in order to work correctly

# Run 1/3 of five loops:

# Check whether the kinase exactly matches the "Gene Symbol"
                    
for d in kinases:
    for r, m in enumerate( kinases_df[ 'Gene_Symbol' ] ):
        if str( d ) == str( m ).upper():
            if d in kinases:
                kinase_dict[ d ] = kinases_df.UniProt_ID[ r ]
                kinases.remove( d )

# Check whether the kinase can be found in "Alternative Gene Name(s)"
                
for e in kinases:
    for s, z in enumerate( kinases_df[ 'Alternative_Gene_Name(s)' ] ):
        if str( e ) in str( z ).upper():
            if e in kinases:
                kinase_dict[ e ] = kinases_df.UniProt_ID[ s ]
                kinases.remove( e )
                
# Check whether kinase plus "_HUMAN" matches the "Entry name"

for a in kinases:
    for o, j in enumerate( kinases_df[ 'Entry_name' ] ):
        if str( a ) + "_HUMAN" == str( j ):
            kinase_dict[ a ] = kinases_df.UniProt_ID[ o ]
            kinases.remove( a )

# Check whether the kinase can be found in "Primary Protein Name"           
            
for b in kinases:
    if b != "ABL": # Matches with the word "probable"
        for p, k in enumerate( kinases_df[ 'Primary_Protein_Name' ] ):
                if str( b ) in str( k ).upper():
                    if b in kinases:
                        kinase_dict[ b ] = kinases_df.UniProt_ID[ p ]
                        kinases.remove( b )

# Check whether the kinase can be found in "Alternative Protein Name(s)"
                        
for c in kinases:
    if c != "LOK": # Matches with the work "Telokin"
        for q, l in enumerate( kinases_df[ 'Alternative_Protein_Name(s)' ] ):
            if str( c ) in str( l ).upper():
                if c in kinases:
                    kinase_dict[ c ] = kinases_df.UniProt_ID[ q ]
                    kinases.remove( c )
                
# Re-running the five loops resolves a further 20 kinase IDs. 
# For example, the previous loops failed to match MAPK13 

# Run 2/3 of five loops

# Check whether the kinase exactly matches the "Gene Symbol"
                    
for d in kinases:
    for r, m in enumerate( kinases_df[ 'Gene_Symbol' ] ):
        if str( d ) == str( m ).upper():
            if d in kinases:
                kinase_dict[ d ] = kinases_df.UniProt_ID[ r ]
                kinases.remove( d )

# Check whether the kinase can be found in "Alternative Gene Name(s)"
                
for e in kinases:
    for s, z in enumerate( kinases_df[ 'Alternative_Gene_Name(s)' ] ):
        if str( e ) in str( z ).upper():
            if e in kinases:
                kinase_dict[ e ] = kinases_df.UniProt_ID[ s ]
                kinases.remove( e )
                
# Check whether kinase plus "_HUMAN" matches the "Entry name"

for a in kinases:
    for o, j in enumerate( kinases_df[ 'Entry_name' ] ):
        if str( a ) + "_HUMAN" == str( j ):
            kinase_dict[ a ] = kinases_df.UniProt_ID[ o ]
            kinases.remove( a )

# Check whether the kinase can be found in "Primary Protein Name"           
            
for b in kinases:
    if b != "ABL": # Matches with the word "probable"
        for p, k in enumerate( kinases_df[ 'Primary_Protein_Name' ] ):
                if str( b ) in str( k ).upper():
                    if b in kinases:
                        kinase_dict[ b ] = kinases_df.UniProt_ID[ p ]
                        kinases.remove( b )

# Check whether the kinase can be found in "Alternative Protein Name(s)"
                        
for c in kinases:
    if c != "LOK": # Matches with the work "Telokin"
        for q, l in enumerate( kinases_df[ 'Alternative_Protein_Name(s)' ] ):
            if str( c ) in str( l ).upper():
                if c in kinases:
                    kinase_dict[ c ] = kinases_df.UniProt_ID[ q ]
                    kinases.remove( c )
                
# The second set of loops fails to pair e.g. "PDHK4" with a UniProt ID. 
# Re-running the loops will add these kinases and their corresponding UniProt
# IDs to the dictionary    

# Run 3/3 of five loops

# Check whether kinase plus "_HUMAN" matches the "Entry name"

# Check whether the kinase exactly matches the "Gene Symbol"
                    
for d in kinases:
    for r, m in enumerate( kinases_df[ 'Gene_Symbol' ] ):
        if str( d ) == str( m ).upper():
            if d in kinases:
                kinase_dict[ d ] = kinases_df.UniProt_ID[ r ]
                kinases.remove( d )

# Check whether the kinase can be found in "Alternative Gene Name(s)"
                
for e in kinases:
    for s, z in enumerate( kinases_df[ 'Alternative_Gene_Name(s)' ] ):
        if str( e ) in str( z ).upper():
            if e in kinases:
                kinase_dict[ e ] = kinases_df.UniProt_ID[ s ]
                kinases.remove( e )
                
# Check whether kinase plus "_HUMAN" matches the "Entry name"

for a in kinases:
    for o, j in enumerate( kinases_df[ 'Entry_name' ] ):
        if str( a ) + "_HUMAN" == str( j ):
            kinase_dict[ a ] = kinases_df.UniProt_ID[ o ]
            kinases.remove( a )

# Check whether the kinase can be found in "Primary Protein Name"           
            
for b in kinases:
    if b != "ABL": # Matches with the word "probable"
        for p, k in enumerate( kinases_df[ 'Primary_Protein_Name' ] ):
                if str( b ) in str( k ).upper():
                    if b in kinases:
                        kinase_dict[ b ] = kinases_df.UniProt_ID[ p ]
                        kinases.remove( b )

# Check whether the kinase can be found in "Alternative Protein Name(s)"
                        
for c in kinases:
    if c != "LOK": # Matches with the work "Telokin"
        for q, l in enumerate( kinases_df[ 'Alternative_Protein_Name(s)' ] ):
            if str( c ) in str( l ).upper():
                if c in kinases:
                    kinase_dict[ c ] = kinases_df.UniProt_ID[ q ]
                    kinases.remove( c )

Several kinases are still not in the dictionary. Translate as many untranslated kinases as possible

In [ ]:
# Make an empty list. We will split each untranslated kinase in two and 
# store the resulting list in this list, along with the original kinase
# name, which will be used in a dictionary later

kinases_2 = []

In [ ]:
# Check for four non-alphanumeric characters in the untranslated kinases
# Split them by these characters if found, store in "kinases_2", and 
# remove from "kinases"
# Attach the original kinase name for use in a dictionary later

# For some reason the following four loops need to be executed four times
# in order to work correctly

# Run 1/4 of four loops:

for i in kinases:
    if " " in i:
        if i in kinases:
            items = i.split( " " )
            items.append(items[ 0 ] + " " + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( i )
        
for j in kinases:
    if "/" in j:
        if j in kinases:
            items = j.split( "/" )
            items.append(items[ 0 ] + "/" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( j )
    
for k in kinases:
    if "_" in k:
        if k in kinases:
            items = k.split( "_" )
            items.append(items[ 0 ] + "_" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( k )

for l in kinases:
    if "-" in l:
        if l in kinases:
            items = l.split( "-" )
            items.append(items[ 0 ] + "-" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( l )

# Several kinases with underscores still remain in the original list

# Run 2/4 of four loops:

for i in kinases:
    if " " in i:
        if i in kinases:
            items = i.split( " " )
            items.append(items[ 0 ] + " " + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( i )
        
for j in kinases:
    if "/" in j:
        if j in kinases:
            items = j.split( "/" )
            items.append(items[ 0 ] + "/" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( j )
    
for k in kinases:
    if "_" in k:
        if k in kinases:
            items = k.split( "_" )
            items.append(items[ 0 ] + "_" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( k )

for l in kinases:
    if "-" in l:
        if l in kinases:
            items = l.split( "-" )
            items.append(items[ 0 ] + "-" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( l )

# Nine splittable kinases remain in the original list

# Run 3/4 of four loops:

for i in kinases:
    if " " in i:
        if i in kinases:
            items = i.split( " " )
            items.append(items[ 0 ] + " " + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( i )
        
for j in kinases:
    if "/" in j:
        if j in kinases:
            items = j.split( "/" )
            items.append(items[ 0 ] + "/" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( j )
    
for k in kinases:
    if "_" in k:
        if k in kinases:
            items = k.split( "_" )
            items.append(items[ 0 ] + "_" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( k )

for l in kinases:
    if "-" in l:
        if l in kinases:
            items = l.split( "-" )
            items.append(items[ 0 ] + "-" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( l )
    
# Three splittable kinases remain in the original list

# Run 4/4 of four loops:

for i in kinases:
    if " " in i:
        if i in kinases:
            items = i.split( " " )
            items.append(items[ 0 ] + " " + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( i )
        
for j in kinases:
    if "/" in j:
        if j in kinases:
            items = j.split( "/" )
            items.append(items[ 0 ] + "/" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( j )
    
for k in kinases:
    if "_" in k:
        if k in kinases:
            items = k.split( "_" )
            items.append(items[ 0 ] + "_" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( k )

for l in kinases:
    if "-" in l:
        if l in kinases:
            items = l.split( "-" )
            items.append(items[ 0 ] + "-" + items[ 1 ])
            kinases_2.append( items )
            kinases.remove( l )

Check the kinases table for the strings in kinases_2

In [ ]:
# Check whether the two substrings from the kinase name
# can be found in the kinase data frame under "Primary Protein Name"
# Add to dictionary if so

# For some reason the following loop needs to be executed twice
# in order to work correctly

for i in kinases_2:
    for n, j in enumerate( kinases_df[ 'Primary_Protein_Name' ] ):
        if i[ 0 ] in j.upper() and i[ 1 ] in j.upper():
            if i in kinases_2:
                kinase_dict[ str( i[ 2 ] )] = kinases_df.UniProt_ID[ n ]
                kinases_2.remove( i )

for i in kinases_2:
    for n, j in enumerate( kinases_df[ 'Primary_Protein_Name' ] ):
        if i[ 0 ] in j.upper() and i[ 1 ] in j.upper():
            if i in kinases_2:
                kinase_dict[ str( i[ 2 ] )] = kinases_df.UniProt_ID[ n ]
                kinases_2.remove( i )

In [ ]:
# Check whether the two substrings from the kinase name
# can be found in the kinase data frame under "Alternative Protein Name(s)"
# Add to dictionary if so

# Ensure only unambiguous translations are added by counting
# the number of matches

# For some reason the following loop needs to be executed twice
# in order to work correctly

for i in kinases_2:
    
    matches = 0
    match = []
    
    for n, j in enumerate( kinases_df[ 'Alternative_Protein_Name(s)' ] ):
        if i[ 0 ] in str( j ).upper() and i[ 1 ] in str( j ).upper():
            matches += 1
            match = kinases_df[ 'UniProt_ID' ][ n ]
    if matches == 1 and i in kinases_2:
        kinase_dict[ str( i[ 2 ] )] = match
        kinases_2.remove( i )     

for i in kinases_2:
    
    matches = 0
    match = []
    
    for n, j in enumerate( kinases_df[ 'Alternative_Protein_Name(s)' ] ):
        if i[ 0 ] in str( j ).upper() and i[ 1 ] in str( j ).upper():
            matches += 1
            match = kinases_df[ 'UniProt_ID' ][ n ]
    if matches == 1 and i in kinases_2:
        kinase_dict[ str( i[ 2 ] )] = match
        kinases_2.remove( i )          

In [ ]:
# Check whether the first substring of the kinase name exactly matches
# "Entry name" minus "_HUMAN" in the kinase data frame

for i in kinases_2:
    for n, j in enumerate( kinases_df[ 'Entry_name' ] ):
        if i[ 0 ] == j[ : -6 ].upper(): # Crop "_HUMAN" from entry name
            # This will be added later
            kinase_dict[ str( i[ 2 ] )] = kinases_df[ 'UniProt_ID' ][ n ]
            kinases_2.remove( i ) 

Add kinase UniProt ID column to data frame

In [ ]:
uniprot_id = []

# If the kinase can be translated, add the UniProt ID to the column
# Otherwise add an empty string

for n, i in enumerate( phosphosite_2_df.kinases ):
    if i in kinase_dict.keys():
        kinase = kinase_dict.get( str( i ) )
        uniprot_id.append( kinase )
    else:
        uniprot_id.append( "" )

uniprot_id = pd.Series(uniprot_id)

phosphosite_2_df = phosphosite_2_df.assign( ACC_ID = uniprot_id )

Using the UniProt ID and the phosphosite amino acid residue information, add a phosphosite ID column to act as a Foreign Key for several different phosphosite-related tables in the database

In [ ]:
phos_id = []

for n, i in phosphosite_2_df.iterrows():
    phos_id.append( phosphosite_2_df.acc[ n ].upper() + "(" + phosphosite_2_df.code[ n ].upper() + str( phosphosite_2_df.position[ n ]) + ")" )

phos_id = pd.Series( phos_id )

phosphosite_2_df = phosphosite_2_df.assign( PHOS_ID = phos_id )

Remove unnecessary columns

In [ ]:
# Remove columns "source" and "entry_date"

phosphosite_2_df = phosphosite_2_df.drop( [ 'source', 'entry_date' ],
                                         axis = 1 )

phosphosite_2_df = phosphosite_2_df.reset_index( drop = True )

Write to CSV

In [ ]:
phosphosite_2_df.to_csv( "phosphosites_2.csv", index = False)